\section{Markov chain Monte Carlo basics}
 When using MCMC methods, we estimate the posterior distribution and the intractable integrals using simulated samples from the posterior distribution.

\subsection{Markov chains}
\begin{Definition}
A Markov chain is a discrete-time stochastic process $\theta_1, \theta_2, ..., \theta_n$ that satisfies the Markov property:
\[
p(\theta_t | \theta_1, ..., \theta_{t-1}) = p( \theta_t | \theta_{t-1} 
\]

\end{Definition}


\subsection{Markov chain convergence theory}
MCMC we will need to focus on Markov chains that converge to a stationary distribution

\begin{Definition}
Irreducible: any state can be reached from any other state with a positive probability in a finite number of steps
\end{Definition}

\begin{Definition}
Periodic with period $k$: the number of steps required to return to the state is always visible by $k$
\end{Definition}

\begin{Definition}
Ergodic: the expected return time to every state is finite.
\end{Definition}

\begin{Definition}
Stationary: $p(\theta_{t+1}, ..., \theta{t+k})$ does not depend on $t$
\end{Definition}

\begin{Definition}
Reversible: $p(\theta_{t}, ..., \theta{t+1})$ does not depend on $t$. A reversible Markov chain is always stationary.
\end{Definition}

\begin{Definition}
Markov chain with transition kernel T satisfies the detailed balance condition if there exists a distribution $\pi(\theta)$ such that
\[
\pi(\theta) T(\theta, \theta') = \pi( \theta') T (\theta', \theta)
\]
\end{Definition}

\subsection{Metropolis-Hastings algorithm}
Metropolis-Hastings sampling works by deriving a Markov chain $(\theta^{(1)}, \theta^{(2)}, ...)$ whose stationary distribution is the target $\pi(\theta)$. A new state $\theta '$ from a proposal density $q(\theta '; \theta^{(t)})$. 
\[
a = \frac{\pi(\theta ')}{\pi (\theta^{t})}\frac{q(\theta^{(t)} ; \theta ')}{q(\theta ' ; \theta^{(t)})}
\]
if $a \leq 1$, then the new state is always accepted

Otherwise, the new state is accepted with probability $a$.

If accepted $\theta^{(t+1)} = \theta '$

If rejected, then we set $\theta^{(t+1)} = \theta^{(t)}$

If the proposal $q$ is symmetric, then:
\[
a = \frac{\pi(\theta ')}{\pi(\theta^{(t)}}
\]

\subsection{Implementation of the Metropolis-Hastings algorithm}

Draw samples from the Laplace distribution $\pi^* (\theta) = \exp (- |\theta| )$, normal proposal $q(\theta ' ; \theta) = N(\theta '; \theta, 1^2)$

Logs of acceptance probability:
\[
\log (a) = \log \pi (\theta ') - \log \pi (\theta^{(t)}) + \log q(\theta^{(t)} ; \theta ') - \log q(\theta '; \theta^{(t)})
\]
if $q$ is symmetric 
\[
\log (a) = \log \pi (\theta ') - \log \pi (\theta^{(t)})
\]


\begin{minted}[breaklines]{Python}
#We use logarithms of distributions

#input x, output -np.abs(x)
#the same as
#def target(x):
#	return -np.abs()x
target = lambda x: -np.abs(x)

import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
 
# Metropolis sampling (symmetric proposal) for given 
#log-target distribution
def mhsample0(x0, n, logtarget, drawproposal):
    x = x0
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        if np.log(npr.rand()) < logtarget(x_prop) - logtarget(x):
            x = x_prop
        xs[i] = x
    return xs

# Testing Laplace target with normal proposal centred 
#around the previous value
npr.seed(42)
x = mhsample0(0.0, 10000, lambda x: -np.abs(x), lambda x: x+npr.normal())
 
# Discard the first half of samples as warm-up
x = x[len(x)//2:]
fig, ax = plt.subplots(1, 2)
#Slicing, every 10th element is plotted
ax[0].plot(x[::10])
h = ax[1].hist(x[::10], 50, normed=True)
plt.show()

\end{minted}

\subsection{MCMC sampling efficiency}
The efficiency of an MCMC sampler depends on the proposal $q(\theta '; \theta)$. 

\begin{minted}[breaklines]{Python}
# Metropolis sampling (symmetric proposal) for given log-target distribution
#The sampler prints the acceptance rate
def mhsample1(x0, n, logtarget, drawproposal):
    x = x0
    xs = np.zeros(n)
    accepts = 0
    for i in range(n):
        x_prop = drawproposal(x)
        if np.log(npr.rand()) < logtarget(x_prop) - logtarget(x):
            x = x_prop
            accepts += 1
        xs[i] = x
    print("Sampler acceptance rate:", accepts/n)
    return xs
 
# Testing Laplace target with a narrow normal proposal
x = mhsample1(0.0, 10000, lambda x: -np.abs(x), lambda x: x+0.1*npr.normal())
 
# Discard the first half of samples as warm-up

x = x[len(x)//2:]
fig, ax = plt.subplots(1, 2)
ax[0].plot(x[::10])
h = ax[1].hist(x[::10], 50, normed=True)
plt.show()

# Testing Laplace target with a well-tuned normal proposal
x = mhsample1(0.0, 10000, lambda x: -np.abs(x), lambda x: x+2.5*npr.normal())
 
# Discard the first half of samples as warm-up

x = x[len(x)//2:]
fig, ax = plt.subplots(1, 2)
ax[0].plot(x[::10])
h = ax[1].hist(x[::10], 50, normed=True)
plt.show()

# Testing Laplace target with a wide normal proposal
x = mhsample1(0.0, 10000, lambda x: -np.abs(x), lambda x: x+50*npr.normal())
 
# Discard the first half of samples as warm-up

x = x[len(x)//2:]
fig, ax = plt.subplots(1, 2)
ax[0].plot(x[::10])
h = ax[1].hist(x[::10], 50, normed=True)
plt.show()
\end{minted}

Too narrow proposals: very high acceptance rates and biased samples
Low acceptance rates: spikes in the histogram of the samples

Acceptance rate for MH: 50\%

Sampling multimodal distributions with MCMC is difficult!

\subsection{Initial best practices}
Initial warm-up are discarded!


\subsection{Exercises}
\subsubsection{Metropolis-Hastings sampling of a 1D target}
1. Write a Metropolis-Hastings sampler to draw samples from the standard zero mean unit variance normal distribution  using the distribution as the proposal. Draw 1000 samples and compute the acceptance rate of your sampler.

2. Plot a normed histogram of the samples you have drawn together with the true density.

3. Plot a trace plot of the samples: a line curve of the samples in order. Do consecutive samples appear independent or are they correlated? Plot a similar curve of the samples when they are permuted to a random order. Do the curves look simiar?

4. Try increasing the number of samples to 10000 and only including every 10th sample in the plots. This is called thinning. (Hint: you can extract every 10th element of a numpy.array x using x[::10].)

5. Try changing the bounds of the proposal distribution such that they remain symmetric about 0, run the sampler and redraw the plots. Can you make the samples appear more independent? Is there a connection between the acceptance rate and the appearance of the plots?
\begin{minted}[breaklines]{Python}
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

#1d symmetric MH sampler
def msample(x0, n, target, drawproposal):
    x = x0
    accepts = 0
    lp = target(x)
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs

# Note that now Q(x',x^(t)) = Q(x^(t), x') so we do not need to include Q(x'; x) term for the acceptance probability
x = msample(0.0, 10000, lambda x: -0.5*x**2, lambda x: x+10*(npr.rand()-0.5))
x = x[len(x)//2:]
h = plt.hist(x[::10], 50, normed=True)
plt.show()
plt.plot(x[::10])
\end{minted}

\subsubsection{Tuning a Metropolis-Hastings sampler}
Implement the sampler and run it 10 times with a number of different values of sigma . Record the acceptance rates and the mean and variance of the samples you obtain. What value of  sigma  gives the most accurate 

\begin{minted}[breaklines]{Python}
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

# return acceptance rate to avoid too many prints
def msample(x0, n, target, drawproposal):
    x = x0
    accepts = 0
    lp = target(x)
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    #print('Acceptance rate:', accepts/n)
    return xs, accepts/n

def sample_normal(nsamples, repeats, drawproposal):
    means = np.zeros(repeats)
    stds = np.zeros(repeats)
    accepts = np.zeros(repeats)
    for i in range(repeats):
        x, accepts[i] = msample(0.0, 10000, lambda x: -0.5*x**2, drawproposal)
        # Drop warm-up samples
        x = x[len(x)//2:]
        means[i] = np.mean(x)
        stds[i] = np.std(x)
    return means, stds, np.mean(accepts)

def check_errors(means, stds, truemean=0.0, truestd=1.0):
    return (np.sqrt(np.mean((means-truemean)**2)), np.sqrt(np.mean((stds-truestd)**2)))

npr.seed(42)
widths = [0.1, 0.3, 1.0, 3.0, 10.0, 30.0]

for w in widths:
    means, stds, accepts = sample_normal(10000, 10, lambda x: x+w*npr.normal())
    meanerr, stderr = check_errors(means, stds)
    print('normal width %.1f' % w, 'acceptance rate', accepts,
          'mean RMSE', meanerr, 'std RMSE', stderr)

for w in widths:
    means, stds, accepts = sample_normal(10000, 10, lambda x: x+w*(npr.random()-0.5))
    meanerr, stderr = check_errors(means, stds)
    print('uniform width %.1f' % w, 'acceptance rate', accepts,
          'mean RMSE', meanerr, 'std RMSE', stderr)

\end{minted}

\subsubsection{Metropolis-Hastings sampling of a higher dimensional target}
Implement a Metropolis-Hastings sampler for multivariate targets and test it for sampling from standard zero-mean unit covariance  dimensional multivariate normal   using as the proposal . Draw 10000 samples and thin by a factor of 10.
Plot trace plots, histograms and pairwise scatter plots of all pairs of variables.

\begin{minted}[breaklines]{Python}
#1
#MH sampler multidimensional symmetric
def d_msample(x0, n, target, drawproposal):
    x = x0
    d = len(x0)
    accepts = 0
    lp = target(x)
    xs = np.zeros([n, d])
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs

def lmvunitnormpdf(x, d=len(x)):
    return -0.5*d*np.log(2*np.pi)-0.5*np.sum(x**2)

target = lambda x: lmvunitnormpdf(x)
# generate a matrix of N(0,1) distributed numbers with same shape as x
propose_draws = lambda x, w: x + w * npr.randn(*x.shape)

w=0.5
d = 2
x = d_msample(np.zeros(d), 10000, target, lambda x: propose_draws(x, w))
x = x[len(x)//2:]

#2
plt.hist2d(x[::10, 0],x[::10, 1], 50, normed=True)
plt.title('w = {}, d = {}'.format(w, d))
plt.show()

plt.plot(x[::10])
plt.title('w = {}, d = {}'.format(w, d))
plt.show()

plt.scatter(x[::10, 0],x[::10, 1], marker = '.')
plt.title('w = {}, d = {}'.format(w, d))
plt.show()

#3
w = 10
x = d_msample(np.zeros(d), 10000, target, lambda x: propose_draws(x, w))
x = x[len(x)//2:]

plt.hist2d(x[::10, 0],x[::10, 1], 50, normed=True)
plt.title('w = {}, d = {}'.format(w, d))
plt.show()

plt.plot(x[::10])
plt.title('w = {}, d = {}'.format(w, d))
plt.show()

plt.scatter(x[::10, 0],x[::10, 1], marker = '.')
plt.title('w = {}, d = {}'.format(w, d))
plt.show()

#4
w = 0.5
d = 5
x = d_msample(np.zeros(d), 10000, target, lambda x: propose_draws(x, w))
x = x[len(x)//2:]

plt.plot(x[::10])
plt.title('w = {}, d = {}'.format(w, d))
plt.show()


w = 10
d = 5
x = d_msample(np.zeros(d), 10000, target, lambda x: propose_draws(x, w))
x = x[len(x)//2:]

plt.plot(x[::10])
plt.title('w = {}, d = {}'.format(w, d))
plt.show()
\end{minted}

\subsubsection{Sampling a multimodal distribution, convergence checking}
1. Plot a contour plot 

2. Write a Metropolis-Hastings sampler for the target using  

3. Plot a scatter plot of the samples and compare it to the contour plot. Is the sampler exploring the density well?

4. Start several chains from different initial values and compare the results. Do the chains converge?

5. Try repeating the experiment.

6. Try repeating the experiment using the same initial points

7. Try to get the sampler to explore the density. Try running the sampler for longer, changing the proposal.

\begin{minted}[breaklines]{Python}
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import scipy.misc as scm

#Multidimensional MH sampler
def d_msample(x0, n, target, drawproposal):
    x = x0
    d = len(x0)
    accepts = 0
    lp = target(x)
    xs = np.zeros([n, d])
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs

def target(x, R):
    means = np.array([[0, 0],[R, R],[-R,R],[R,-R],[-R,-R]])
    K = means.shape[0]
    logp = -0.5*np.sum((x[np.newaxis,:]-means)**2, 1)
    return scm.logsumexp(logp)

#Countour plotting
R = 6.0
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.zeros(X.shape)
for i, x_ in enumerate(x):
    for j, y_ in enumerate(y):
        Z[i,j] = target(np.array([x_, y_]), R)
        
plt.contour(X, Y, Z)

R = 6.0

x = d_msample(np.array([0.0, 0.0]), 10000, lambda x: target(x, R), lambda x: x+npr.randn(2))
x = x[len(x)//2:]
plt.plot(x[:,0], x[:,1], '.')

R = 3.0

x = d_msample(np.array([0.0, 0.0]), 10000, lambda x: target(x, R), lambda x: x+npr.randn(2))
x = x[len(x)//2:]
plt.plot(x[:,0], x[:,1], '.')

R = 100.0

x = d_msample(np.array([0.0, 0.0]), 10000, lambda x: target(x, R), lambda x: x+npr.randn(2))
x = x[len(x)//2:]
plt.plot(x[:,0], x[:,1], '.')

R = 6.0

x = d_msample(np.array([0.0, 0.0]), 10000, lambda x: target(x, R), lambda x: x+npr.randn(2))
x = x[len(x)//2:]
plt.plot(x[:,0], x[:,1], '.')

R = 6.0

npr.seed(42)
# Increase step length and iteration count
x = d_msample(np.array([0.0, 0.0]), 100000, lambda x: target(x, R), lambda x: x+2*npr.randn(2))
x = x[len(x)//2:]
plt.plot(x[:,0], x[:,1], '.')
\end{minted}

\section{Bayesian inference with MCMC}
The most important use of MCMC sampling in statistics is in Bayesian inference and drawing samples from the posterior distribution

\subsection{Bayesian inference}
Given the observations $D$, a model $p(D | \theta)$ with parameters $\theta$, and prior probability $p(\theta)$ of the parameters, the posterior probability distribution is 
\[
p(\theta | D) = \frac{p(D | \theta) p(\theta)}{p(D)} = \frac{p(D | \theta) p(\theta)}{\int_{\theta}p(D|\theta)p(\theta)d\theta}
\]
However, the integral or the marginal likelihood is hard to compute, and cannot be used directly. But in terms of the posterior distribution, it is just a scalar normaliser.

If the used prior is conjugate to the likelihood, then the posterior will have the same functional form. 

Let us consider Bayesian inference of the mean $\mu$ of normally distributed data $D=(x_1, ..., x_n)$ with known variance $\sigma_x^2$.

Assuming the observations are independent 
\[
p(D | \theta) = \prod_{i=1}^n p(x_i|\mu) = 
\prod_{i=1}^n N(x_i; \mu, \sigma_x^2)
\]

We use a normal prior
\[
p(\mu) = N(\mu; \mu_0, \sigma_0^2)
\]

Then we get 
\[
p(\mu | D) \propto \prod_{i=1}^n N(x_i; \mu, \sigma_x^2) N(\mu; \mu_0, \sigma_0^2)
\]

\subsection{Sampling the posterior with MCMC}
Metropolis-Hastings algorithm can work with an unnormalised density

We define the target as
\[
\log \pi ^* (\theta) = \log p(D | \mu) + \log p(\mu)
\]

And use the normal proposal $q(\theta'; \theta) = N(\theta'; \theta, \sigma_{prop}^2$

\begin{minted}{Python}
# Define the normal pdf. Note parameters: mean, standard deviation
# Note the sum to allow evaluation for a data set at once
def lnormpdf(x, mu, sigma):
    return np.sum(-0.5*np.log(2*np.pi) - np.log(sigma) - 
    0.5 * (x-mu)**2/sigma**2)
 
# Define the target log-pdf as a sum of likelihood and prior terms
def target(mu, data, sigma_x, mu0, sigma0):
    return lnormpdf(data, mu, sigma_x) + lnormpdf(mu, mu0, sigma0)
 
# Simulate n=100 points of normally distributed data about mu=0.5
n = 100
data = 0.5 + npr.normal(size=n)
 
# Set the prior parameters
sigma_x = 1.0
mu0 = 0.0
sigma0 = 3.0
 
# Run the sampler
x = mhsample1(0.0, 10000,
              lambda mu: target(mu, data, sigma_x, mu0, sigma0),
              lambda x: x+0.2*npr.normal())
 
# Discard the first half of samples as warm-up
x = x[len(x)//2:]
# Plot a histogram of the posterior samples
plt.hist(x, 30, normed=True)
# Compare with analytical result from the above example
tt = np.linspace(0.1, 0.9, 50)
m_post = sigma0**2 * np.sum(data) / (n*sigma0**2 + sigma_x**2)
s2_post = 1/(n/sigma_x**2 + 1/sigma0**2)
y = np.array([np.exp(lnormpdf(t, m_post, np.sqrt(s2_post))) for t in tt])
plt.plot(tt, y)
plt.show()
\end{minted}

\subsection{Sampling bounded variables}
Many probabilistic models contain variables that have some constraints: variances need to be positive and point probabilities need to be in the interval [0,1].

Variables $\theta_I$ with bounds $\theta_I \in B$

\subsubsection{Zero likelihood approach}
Convergence to the correct distribution: $P^*(\theta) = 0$, when $\theta_I$ not in $B$. 

\subsubsection{Proposal that respects the bounds}
$\theta_i > 0$. One way to enforce the constraint is to propose $\theta ' = |\theta_0'|$, where $\theta_0'$ follows some proposals $q_0(\theta_0'; \theta)$. 

\subsubsection{Transformations to unbounded variables}
Samplers for bounded variables can also be implemented using a change of variables through a transformation to unbounded variables.

\begin{enumerate}
    \item If $\theta_i > 0$, choose $g(\phi) = \exp(\theta)$
    \item If $\theta_i \in (0,1)$, choose $g(\phi) = \frac{1}{(1 + \exp(-\phi))}$: the logistic transformation
    \item If $\theta_i \in (0,1), i \in I$ with $\sum_{i \in I} \theta_i = 1$, $g_i(\phi) = \frac{\exp(\phi_i)}{\sum_{i' \in I} \exp (\phi_{i'}}$
\end{enumerate}

\begin{minted}{Python}
import scipy.integrate as si
import numpy as np

#g(phi) = exp(phi)

def p_theta(theta, l):
    return l * np.exp(-l * theta)
    

def p_phi(phi, l):
    return l * np.exp(-l * np.exp(phi)) * np.exp(phi)
    

# Note the different bounds for the integrals!
print('p_theta normaliser:', si.quad(lambda x: p_theta(x, 2.0), 0, np.inf))

print('p_phi normaliser:  ', si.quad(lambda x: p_phi(x, 2.0), -100, 100))
# Note: setting bounds to infinity would give an error, but
# this range is sufficient in practice

# Note the different bounds for the integrals!
print('p_theta probability:', si.quad(lambda x: p_theta(x, 2.0), 
1, np.exp(1)))

print('p_phi probability:  ', si.quad(lambda x: p_phi(x, 2.0), 0, 1))
\end{minted}

\subsection{Exercises}
\subsubsection{Metropolis-Hastings sampling for the posterior distribution of a probabilistic model}
Estimating the mean $\mu$ of a normal distribution for some observed data $X=(x_i)_{i=1}^n$ that are assumed to be independent given $\mu$.

$p(x_i | \mu) = N(x_i ; \mu, \sigma^2)$

Our goal is $p(\mu | X)$:
\[
p(\mu | X) = \frac{p(X | \mu) p(\mu)}{p(X)} = \frac{\prod_{i=1}^n p(x_i | \mu) p(\mu)}{p(X)}
\]

1. Assume $p(\mu) = N(\mu; 0, \sigma_0^2)$. Implement Metropolis-Hastings to sample from $p(\mu | X)$ using a normal proposal and $\sigma_0^2 = 100$. 

2. Compare with exact solution

3. Repeat with $\sigma_0^2 = 1$

4. Try $p(\mu) = Laplace(\mu ; 0, b_0) = \frac{1}{2b_0}\exp{(-|\mu|/b_0)}$ with $b_0 = 10$

\begin{minted}{Python}
\%matplotlib inline
#Import packages
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt

import scipy.integrate

#Import data as a numpy.array
data = pd.read_csv('http://www.helsinki.fi/~ahonkela/teaching/
compstats1/toydata.txt', sep='\t', header=None)
data = data.values
data = np.array(data[:,0])

#Known parameters
sigma0 = 10
sigma = np.sqrt(2)

#1D Metropolis-Hastings sampler with acceptance rate. target and drawproposals are functions with input x, output f(x)
def msample(x0, n, target, drawproposal):
    x = x0
    accepts = 0
    lp = target(x)
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs

# Note: sigma = standard deviation, not variance
#Logarithm of a multivariate standard normal distribution
def lnormpdf(x, mu, sigma):
    return -0.5*np.log(2*np.pi) - np.log(sigma) - 0.5*(x-mu)**2/sigma**2

# check that the density integrates to 1
print(scipy.integrate.quad(lambda x: np.exp(lnormpdf(x, 0, 10)), -100, 100))

# Target function input: mu (the parameter) output: probability p(mu | data) 
# p(mu | data) = p(data | mu) * p(mu)
target = lambda mu: np.sum(lnormpdf(data, mu, sigma)) + lnormpdf(mu, 0, sigma0)
#Run the sampler. Target p(\mu) and proposal q(x)
x = msample(1.0, 100000, target, lambda x: x+0.2*npr.normal())
#Discard first half of samples
x = x[len(x)//2:]

print(np.mean(x), np.std(x))

#Plotting data
n = len(data)
t = np.linspace(3.5, 4.5, 100)
plt.hist(x, 30, normed=True)
#Plotting exact solution
plt.plot(t, np.exp(lnormpdf(t, sigma0**2/(sigma**2/n + sigma0**2) * np.mean(x), np.sqrt(1/(1/sigma0**2 + n/sigma**2)))))
plt.show()

print(np.mean(x), np.std(x))

sigma0 = 1
target2 = lambda mu: np.sum(lnormpdf(data, mu, sigma)) + lnormpdf(mu, 0, sigma0)
x = msample(1.0, 100000, target2, lambda x: x+0.2*npr.normal())
x = x[len(x)//2:]
print(np.mean(x), np.std(x))

n = len(data)
t = np.linspace(3.5, 4.5, 100)
plt.hist(x, 30, normed=True)
plt.plot(t, np.exp(lnormpdf(t, sigma0**2/(sigma**2/n + sigma0**2) * np.mean(data), np.sqrt(1/(1/sigma0**2 + n/sigma**2)))))
plt.show()
\end{minted}

\subsubsection{Metropolis-Hastings sampling of constrained parameters using transformations}
Bayesian inference of the variance of normally distributed data. We use transformations to enforce the positivity of the variance parameter

\[
p(x_i | \sigma) = N(x_i ; 0, \sigma^2)
\]
\[
p(\sigma) = Exponential(\sigma; 1) = e^{-\sigma}
\]

We will apply transformation $\sigma = g(y) = \exp(y)$ as $\sigma \in R^{+}$, $y \in R$.
\[
p_Y(y) = p_{\sigma}(\sigma) |\frac{dg}{dy}|
\]

1. Express and plot the prior density $p(\sigma)$ as a function $y = \log \sigma$ 
2. Evaluate $\int_{-\infty}^{\infty}f_Y(y)dy$
3. Generate a data set $x_i ~ N(0, 2^2)$
4. Implement MH sampler $y = \log \sigma$ when $\log P(y) = \sum_{i=1}^{10} \log p(x_i | y) + \log p(y)$ using $Q(x'; x) = N(x'; x, 1)$ as the proposal
5. Plot a histogram

\begin{minted}{Python}
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import scipy.integrate

#Using the change of variables formula pY(y) = ps(s)|dg/dy|
#log(e^{-exp(y)}*e^y) = -e^y + y
def logf(y):
    return -np.exp(y) + y

t = np.linspace(-5, 5, 50)
#Plot the prior density
plt.plot(t, np.exp(logf(t)))
#Checking density by integrating
print(scipy.integrate.quad(lambda x: np.exp(logf(x)), -30, 30))

#1D log of standard normal, independent x_i!
def lnormpdf(x, sigma):
    return np.sum(-0.5*np.log(2*np.pi) - np.log(sigma) - 0.5*x**2/sigma**2)

#Metropolis-Hastings sampler
def msample(x0, n, target, drawproposal):
    x = x0
    accepts = 0
    lp = target(x)
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs

#Generate a data set of 10 points
d = npr.normal(0, 2, 10)

# Note that now Q(x',x^(t)) = Q(x^(t), x') so we do not need 
#to include Q(x'; x) term for the acceptance probability
x = msample(0.0, 10000, lambda y: logf(y) + lnormpdf(d, np.exp(y)), lambda y: y+npr.randn())
#Discarding the first half
x = x[len(x)//2:]

plt.show()
plt.hist(np.exp(x[::10]), 50, normed=True)
print('Data std', np.std(d))
print('posterior mean of sigma', np.mean(np.exp(x)))
\end{minted}

\subsubsection{MCMC with an asymmetric proposal}
The target distribution:
\[
p(n) = \begin{cases}
1/21 & when 1 \leq n \leq 21 \\
0 & otherwise
\end{cases}
\]
The proposal: 
\[
q(n';n) = \begin{cases}
1/2 & when |n' - n| = 1 \\
0 & otherwise
\end{cases}
\]

1. Implement the sampler and run it for 100 000 iterations

2. Use an alternative proposal
\[
q'(n';n) = \begin{cases}
1 & when (n, n') \in \{(1,2), (21,20)\} \\
1/2 & when 2 \leq n \leq 20 and |n' - n| = 1 \\
0 & otherwise
\end{cases}
\]
but do not include the term in the acceptaqnce ratio

3. Use the full acceptance rule

\begin{minted}{Python}
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

#Normal symmetric 1D Metropolis-Hastings sampler
def msample(x0, n, target, drawproposal):
    x = x0
    accepts = 0
    lp = target(x)
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs

def logtarget(x):
    if x >= 1 and x <= 21:
        return -np.log(21)
    else:
        return -np.inf

def proposal1(x):
    return x+npr.choice(np.array([-1, 1]))
    
npr.seed(42)
x = msample(3, 30000, logtarget, proposal1)
plt.plot(x)
plt.show()
h = plt.hist(x, 21)

#2. 
def proposal2(x):
    if x == 1:
        return 2
    if x == 21:
        return 20
    return x+npr.choice(np.array([-1, 1]))

npr.seed(42)
x = msample(3, 30000, logtarget, proposal2)
plt.plot(x)
plt.show()
h = plt.hist(x, 21)

#3. 
#Asymmetric Metropolis-Hastings sampler
def mhsample(x0, n, target, proposal, drawproposal):
    x = x0
    accepts = 0
    lp = target(x)
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp + proposal(x_prop, x) - proposal(x, x_prop):
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs

def logproposal2(x, y):
    if x == 1 and y == 2:
        return 0
    if x == 21 and y == 20:
        return 0
    if np.abs(x - y) == 1:
        return np.log(0.5)
    else:
        return -np.inf

npr.seed(43)
x = mhsample(3, 30000, logtarget, logproposal2, proposal2)
plt.plot(x)
plt.show()
h = plt.hist(x, 21)
\end{minted}

\subsubsection{Gibbs sampling}
Special form of multivariate Metropolis-Hastings sampling where a vector of variables $\theta = (\theta_1, \dots, \theta_n)$ is updated cyclically. For each update. $\theta_i$ is drawn from the exact conditional distribution $p(\theta_i | \theta_{\i}$ given the other variables $\theta_{\i}$.

\begin{minted}{Python}
def gibbs2d(x0, n, conditional_draw):
    d = 2
    x = x0
    xs = np.zeros([n, d])
    for t in range(n):
        for i in range(d):
            x[i] = conditional_draw(x[1-i])
        xs[t]=x
    return xs

rho = 0.8
conditional_draw = lambda x: npr.normal(loc=rho*x, scale=np.sqrt(np.sqrt(1-rho**2)))
x = gibbs2d(np.zeros(2), 10000, conditional_draw)
plt.hist2d(x[::10, 0],x[::10, 1], 50, normed=True)
plt.show()
\end{minted}

\section{MCMC diagnostics and sampling multimodal distributions}

\subsection{MCMC diagnostics}
Impossible to prove that a MCMC have converged, but a number of diagnostics have been developed to detect if it has not converged.  Also efficiency is important

\subsubsection{Convergence diagnostics}
Starting $m$ chains from widely dispersed starting points end up producing similar results

Scale reduction $\hat{R}$ that measures the factor by which the scale of the current distribution might be reduced if the simulations were continued until $n \to \infty$ . Based on within-chain variance $W$ and between-chain variance $B$ 

Assuming $\theta_{ij}$ is the $i$th sample from chain $j$ (out of $m$)
\[
B = \frac{n}{m-1}\sum_{j=1}^m (\theta_{:j} - \theta_{::})^2 
\]
\[
\theta_{:j} = \frac{1}{n} \sum_{i = 1}^n \theta_{iJ},  \theta_{::} = \frac{1}{m} \sum_{j=1}^m \theta_{:j}
\]
\[
W = \frac{1}{m}\sum_{j=1}^m s_j^2, s_j^2 = \frac{1}{n-1}\sum_{i=1}^n (\theta_{ij} - \theta_{:j})^2
\]
\[
Var(\theta) = \frac{n-1}{n}W + \frac{1}{n}B, \hat{R} = \sqrt{\frac{Var(\theta)}{W}}
\]
Threshold 1.1 is commonly used to consider a chain as having converged.

Split-$\hat{R}$ : split the chains to two halves and consider these as independent chains 

\subsubsection{Diagnostics of sampling efficiency}
Measured using autocorrelations of the samples, which measure the correlation of neighbouring samples after a given lag. 

ESS: effective sample size, measure of how many independent samples a given sample is worth.

Autocorrelation $p_k$ of a wide-sense stationary process $\theta_1, \theta_2, \dots, $with lag $k$ is defined as
\[
p_k = \frac{E[(\theta_i - \mu)(\theta_{i+k} - \mu)]}{\sigma^2}, k=0,1,2, \dots, 
\]$M$ samples
\[
ESS = \frac{M}{1 + 2 \sum_{k=1}^{\infty}p_k}
\]
Long lags can be unreliable! Then truncate the sum in the denominator

\subsection{Good practices for MCMC sampling}
\begin{enumerate}
\item  Run three or more chains from very different points
\item Discard warm-up or burn-in period
\item Tune the proposal
\item Check for convergence by using the $\hat{R}$ statistic
\item Mix the samples from all chains for final analysis
\item Compare your inference with results from cimpler models or approximations
\end{enumerate}

\subsection{Improving MCMC convergence}
\subsubsection{Blocking for high-dimensional distributions}
Acceptance rates for proposals naturally go down as the dimensionality of the problem increases. If a scalar proposal has a probability $p$ of being accepted, $d$ independent such proposals will have the probability $p^d$

Blocking: divide the parameters to subsets and update each subset alone in turn.

Gibbs sampler: implement an extreme form of blocking where each variable is updated i turn. Blocking can lead to inefficient sampling if variables in different blocks are strongly correlated. 

\subsubsection{Parallel tempering for MCMC for multimodal distributions}
Sampling multimodal distributions with MCMC is hard. Combining small-scale local exploration with large global jumps

Multimodal distributions: run several MCMC chains in parallel!

Instead of sampling a single target $\pi(\theta) $, we sample $\pi(\theta)^{\beta}$ for a range of values $0 \leq \beta \leq 1$. Smooth transition from the actual target to a uniform distribution. 

Bayesian models: $\pi_{\beta}(\theta) = p(\theta)p(D | \theta)^{\beta}$

Given a set of temperature values $B = (\beta_1 = 1, \dots, \beta_b = 0)$ , we can define an augmented system over 
\[
\Theta = (\theta^{(1)}, \dots, \theta^{(b)})
\]
with 
\[
\pi(\Theta) = \prod_{i=1}^b \pi_{\beta_i}(\theta^{(i)})
\]
Following the blocking approach, we introduce a sequence of proposals that are considered and accepted/rejected in sequence:

1. Standard MH proposals for each $\pi_{\beta_i}(\theta^{(i)})$
2. A proposal to swap the states of neighbouring chains

Acceptance probability
\[
a = \frac{\pi_{\beta_i}(\theta^{(i+1)})\pi_{\beta_i+1}(\theta^{(i)})}{\pi_{\beta_i}(\theta^{(i)})\pi_{\beta_i+1}(\theta^{(i+1)})}
\]
\subsubsection{Parallel tempering example}
Multimodal target $\pi(\theta) = \exp(-\gamma (\theta^2 - 1)^2 )$. We use a normal proposal $q_{\beta}(\theta'; \theta) = N(\theta'; \theta, \sigma_{prop, \beta}^2)$ with $\sigma_{prop, \beta}^2 = \frac{1}{\beta}0.1^2$

\begin{minted}{Python}
import scipy.integrate
 
def ltarget(theta, gamma):
    return -gamma*(theta**2-1)**2
 
# Find the normaliser of the target for visualisation
Z = scipy.integrate.quad(lambda theta: np.exp(ltarget(theta, 64.0)), -2, 2)
 
npr.seed(42)
theta = mhsample1(1.0, 10000, lambda theta: ltarget(theta, 64.0),
              lambda theta: theta + 0.1*npr.normal())
              
theta = theta[len(theta)//2:]
h = plt.hist(theta, 50, normed=True)
t = np.linspace(-1.2, 1.2, 200)
plt.plot(t, np.exp(ltarget(t, 64.0)) / Z[0])
plt.show()

print(np.mean(theta))

betas = np.logspace(-3, 0, 5)
print(betas)

def pt_target(theta, beta, gamma):
    return beta * ltarget(theta, gamma)
 
def pt_msample(theta0, n, betas, target, drawproposal):
    CHAINS = len(betas)
    accepts = np.zeros(CHAINS)
    swapaccepts = np.zeros(CHAINS-1)
    swaps = np.zeros(CHAINS-1)
    # All variables are duplicated for all the chains
    theta = theta0 * np.ones(CHAINS)
    lp = np.zeros(CHAINS)
    thetas = np.zeros((n, CHAINS))
    for j in range(CHAINS):
        lp[j] = target(theta[j], betas[j])
    for i in range(n):
        # Independent moves for every chain, MH acceptance
        for j in range(CHAINS):
            theta_prop = drawproposal(theta[j], betas[j])
            l_prop = target(theta_prop, betas[j])
            if np.log(npr.rand()) < l_prop - lp[j]:
                theta[j] = theta_prop
                lp[j] = l_prop
                accepts[j] += 1
        # Swap move for two chains, MH acceptance:
        j = npr.randint(CHAINS-1)
        h = target(theta[j+1],betas[j])+target(theta[j],betas[j+1]) - lp[j] - lp[j+1]
        swaps[j] += 1
        if np.log(npr.rand()) < h:
            # Swap theta[j] and theta[j+1]
            temp = theta[j]
            theta[j] = theta[j+1]
            theta[j+1] = temp
            lp[j] = target(theta[j], betas[j])
            lp[j+1] = target(theta[j+1], betas[j+1])
            swapaccepts[j] += 1
        thetas[i,:] = theta
    print('Acceptance rates:', accepts/n)
    print('Swap acceptance rates:', swapaccepts/swaps)
    return thetas
 
npr.seed(42)
betas = np.logspace(-3, 0, 5)
theta = pt_msample(1.0, 10000, betas,
                   lambda theta, beta: pt_target(theta, beta, 64.0),
                   lambda theta, beta: theta + 0.1/np.sqrt(beta)*npr.normal())
                   
theta = theta[len(theta)//2:]
h = plt.hist(theta[:,-1], 50, normed=True)
t = np.linspace(-1.2, 1.2, 200)
plt.plot(t, np.exp(ltarget(t, 64.0)) / Z[0])
plt.show()

print(np.mean(theta[:,-1]))

N = len(betas)
fix, ax = plt.subplots(N, 1, figsize=[6.4, 9.6])
for i in range(N):
    ax[i].set_title('beta = %.2f' % betas[i])
    ax[i].plot(theta[:,i])
    plt.subplots_adjust(hspace=0.4)
plt.show()
\end{minted}

\subsection{More MCMC convergence theory}
Sampling expectations converge almost surely to corresponding expectations under the stationary distribution
\[
\lim_{N \to \infty} = \frac{1}{N} \sum_{i=1}^N h(\theta_i) \to E_{\pi}[h(\theta)] = \int h(\theta) \pi(\theta) d\theta
\]assuming $\int |h(\theta)|\pi(\theta)d\theta < \infty$
\begin{Theorem}
The Metropolis-Hastings acceptance probability for proposal $q(\theta';\theta)$ and target $\pi(\theta)$,
\[
f(\theta' | \theta) = min \Bigg( 1, \frac{\pi(\theta')q(\theta; \theta')}{\pi(\theta)q(\theta';\theta)}\Bigg)
\]
defines a Markov chain with the transition kernel $T(\theta, \theta') = f(\theta' | \theta)q(\theta'; \theta)$. This chain satisfies the detailed balance condition.  
\end{Theorem}

\subsection{Exercises}
\subsubsection{MCMC with a fixed proposal}
MCMC sampling can also be used so that the proposal $q(x';x)$ is independent of $x$. Metropolis-Hastings sampler for the $N(0,1)$ distribution using $Laplace(0,1)$ as the fixed proposal.

\begin{minted}{Python}
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

#The Laplace function
def laplacetarget(x):
    return -np.log(2) - np.abs(x)

#The standard normal
def normaltarget(x):
    return -0.5*(x**2)

#Asymnmetric MH sampler
def mhsample(x0, n, target, proposal, drawproposal):
    x = x0
    accepts = 0
    lp = target(x)
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp + proposal(x_prop, x) - proposal(x, x_prop):
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs

npr.seed(42)

#The proposal q(x';x) is independent of x! 
#lambda x,y: laplacetarget(y)
x = mhsample(1.0, 30000, normaltarget, lambda x,y: laplacetarget(y), lambda x: npr.laplace())
x = x[len(x)//2:]
plt.plot(x)
plt.show()
h = plt.hist(x, 100)
print('sampled', np.mean(x**2), np.mean(x**4))
y = npr.laplace(size=10000)
print('Laplace', np.mean(y**2), np.mean(y**4))
y = npr.normal(size=10000)
print('normal', np.mean(y**2), np.mean(y**4))

\end{minted}

\subsubsection{Parallel tempering MCMC for multimodal distributions}
Bimodal distribution $\pi(\theta) = \exp(-\gamma(\theta^2 - 1)^2)$

\begin{minted}{Python}
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

#Normal symmetric Metropolis-Hastings sampler
def msample(x0, n, target, drawproposal):
    x = x0
    accepts = 0
    lp = target(x)
    xs = np.zeros(n)
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs[len(xs)//2:]
    
def target(x, gamma):
    return -gamma*(x**2-1)**2

npr.seed(42)
x = msample(1.0, 10000, lambda x: target(x, 16.0), lambda x: x + 1.0*npr.normal())
print(np.mean(x))
h = plt.hist(x, 50)
plt.show()
x = msample(1.0, 10000, lambda x: target(x, 32.0), lambda x: x + 0.5*npr.normal())
print(np.mean(x))
h = plt.hist(x, 50)
plt.show()
x = msample(1.0, 10000, lambda x: target(x, 64.0), lambda x: x + 0.2*npr.normal())
print(np.mean(x))
h = plt.hist(x, 50)
plt.show()

#Parallel tempering target
def par_target(x, beta, gamma):
    return beta*target(x, gamma)

#Parallel tempering MH sampler
def par_msample(x0, n, betas, target, drawproposal):
    CHAINS = len(betas)
    accepts = np.zeros(CHAINS)
    swapaccepts = np.zeros(CHAINS-1)
    swaps = np.zeros(CHAINS-1)
    x = x0 * np.ones(CHAINS)
    lp = np.zeros(CHAINS)
    xs = np.zeros((n, CHAINS))
    for j in range(CHAINS):
        lp[j] = target(x[j], betas[j])
    for i in range(n):
        # Independent moves for every chain
        for j in range(CHAINS):
            x_prop = drawproposal(x[j], betas[j])
            l_prop = target(x_prop, betas[j])
            if np.log(npr.rand()) < l_prop - lp[j]:
                x[j] = x_prop
                lp[j] = l_prop
                accepts[j] += 1
        # Swap move for two chains:
        j = npr.randint(CHAINS-1)
        h = target(x[j+1],betas[j])+target(x[j],betas[j+1]) - lp[j] - lp[j+1]
        swaps[j] += 1
        if np.log(npr.rand()) < h:
            temp = x[j]
            x[j] = x[j+1]
            x[j+1] = temp
            lp[j] = target(x[j], betas[j])
            lp[j+1] = target(x[j+1], betas[j+1])
            swapaccepts[j] += 1
        xs[i,:] = x
    print('Acceptance rates:', accepts/n)
    print('Swap acceptance rates:', swapaccepts/swaps)
    return xs[len(xs)//2:]
 
 npr.seed(42)
betas = np.logspace(-3, 0, 5)
x = par_msample(1.0, 10000, betas,
                 lambda x, beta: par_target(x, beta, 64.0),
                 lambda x, beta: x + 0.1/np.sqrt(beta)*npr.normal())
h = plt.hist(x[:,-1], 50)
plt.show()
print(np.mean(x[:,-1]))
\end{minted}

\subsubsection{Blocking in MCMC}
We will sample a $d = 100$ dimensional multivariate normal with two covariances:
a) $\Sigma = I_d$
b) $\Sigma = AA^T + \epsilon I_d$where all entries of $A \in R^{d \times k}$ follow $N(0,1)$

1. Sample without blocking
2. Sample with blocking

\begin{minted}{Python}
import numpy as np
import numpy.random as npr

#A blocked normal proposal
class blocked_normal_proposal:
    """A class for a blocked normal proposal.
      p = blocked_normal_proposal(d, blocks)
    creates a proposal object yielding d-dimensional normal random numbers
    where each of the "blocks" blocks in turn follows N(0, I) when calling
      p.draw()
    """
    def __init__(self, d, blocks):
        self.d = d
        self.blocks = blocks
        self.curblock = 0
    
    def next_block_indices(self):
        I = np.zeros(self.d, np.bool)
        I[((self.curblock*self.d)//self.blocks):(((self.curblock+1)*self.d)//self.blocks)] = True
        self.curblock = (self.curblock + 1) % self.blocks
        return I
    
    def draw(self):
        res = np.zeros(self.d)
        I = self.next_block_indices()
        res[I] = npr.normal(size=np.sum(I))
        return res

# Use this to get help:
# ? blocked_normal_proposal
p = blocked_normal_proposal(6, 3)
print(p.draw())
print(p.draw())
print(p.draw())
print(p.draw())

import numpy as np
import numpy.random as npr

def d_msample(x0, n, target, drawproposal):
    x = x0
    d = len(x0)
    accepts = 0
    lp = target(x)
    xs = np.zeros([n, d])
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        if  np.log(npr.rand()) < l_prop - lp:
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    print('Acceptance rate:', accepts/n)
    return xs
    
 def mvnormtarget(x):
    return -0.5*np.sum(x**2)

def sample_nd(d, samples=100000, width=1.0):
    return d_msample(np.ones(d), samples, mvnormtarget, lambda x: x+width*npr.normal(size=d))

npr.seed(42)
x = sample_nd(1000, 10000, 0.1)
np.mean(np.mean(x, 0)**2)

def sample_nd_blocked(d, blocks, samples=100000, width=1.0):
    p = blocked_normal_proposal(d, blocks)
    return d_msample(np.ones(d), samples, mvnormtarget, lambda x: x+width*p.draw())
    
npr.seed(42)
x = sample_nd_blocked(1000, 500, 10000, 2.0)
print(np.mean(np.mean(x, 0)**2))

import numpy as np
import numpy.random as npr
import numpy.linalg as nlg
import scipy.linalg as slg

A = npr.normal(size=(100, 2))
Sigma = A @ A.T + 0.1*np.eye(100)
M = nlg.cholesky(Sigma)

def mvnormtarget2(x):
    return -0.5*np.sum(slg.solve_triangular(M, x, lower=True)**2)

def sample_nd2(d, samples=100000, width=1.0):
    x = d_msample(np.ones(d), samples, mvnormtarget2, lambda x: x+width*npr.normal(size=d))
    return x[len(x)//2:]

def sample_nd2_blocked(d, blocks, samples=100000, width=1.0):
    p = blocked_normal_proposal(d, blocks)
    x = d_msample(np.ones(d), samples, mvnormtarget2, lambda x: x+width*p.draw())
    return x[len(x)//2:]

npr.seed(42)
x = sample_nd2(100, 10000, 0.05)
print(np.mean(np.mean(x, 0)**2))
plt.plot(x[:,20])

npr.seed(42)
x = sample_nd2_blocked(100, 50, 10000, 0.8)
print(np.mean(np.mean(x, 0)**2))
plt.plot(x[:,20])
\end{minted}


\section{MCMC for model comparison and other Monte Carlo techniques}

\subsection{Importance sampling}
Importance sampling is another general purpose tool for evaluating expectations over a difficult density $f(x)$ by drawing samples from an easy-to-sample surrogate distribution $g(x)$ and weighting them by the ratio of the true density  

\[
I_h = E_{f(x)}[h(x)] = \int h(x) f(x) dx = \int h(x) \frac{f(x)}{g(x)} g(x) dx = E_{g(x)}\Bigg[ h(x) \frac{f(x)}{g(x)}\Bigg]
\]
The corresponding Monte Carlo approximation is 
\[
I_h \approx \frac{1}{n} \sum_{i=1}^n h(x_i) \frac{f(x_i)}{g(x_i)} = \frac{1}{n} \sum_{i=1}^n h(x_i) w_i
\]
where $x_i \sim g(x)$

It is necessary to have $g(x) = 0 \implies h(x)f(x)= 0$

It is required that 
\[
E_g \Bigg[h^2(x) \frac{f^2(x)}{g^2(x)} \Bigg] = \int h^2(x) \frac{f^2(x)}{g(x)}dx < \infty
\]
which is guaranteed for example if 
\[
Var_f[h(X)] < \infty, and \frac{f(x)}{g(x)} \leq M, \forall x  \textrm{ for some } M>0
\]
\subsection{Bayesian model comparison}
In many inference problems we do not have a single model known beforehand but we want to consider multiple possible models

Marginal likelihood measures the fit of the model $M_i$ to data $D$
\[
p(D | M_i) = \int_{\Theta} p(D | \theta, M_i) p(\theta | M_i) d\theta
\]
\subsubsection{Model comparison and Bayes factors}
Bayes rule
\[
p(M_i | D) = \frac{p(D |M_i) p(M_i)}{\sum_i p(D |M_i) p(M_i)}
\]
Bayes factor: $B_{12} = \frac{p(M_1 | D)}{p(M_2 |D)}\frac{p(D | M_1)}{p(D | M_2)}$

\subsubsection{Caveats}
Depends on the prior

\subsection{Marginal likelihood evaluation using thermodynamic integration}
Evaluating marginal likelihood with MCMC is non-trivial, results in a very high variance of the results. Use thermodynamic integration!

Drawing samples from
\[
\pi_{\beta}^* (\theta) = p(D | \theta)^{\beta} p(\theta), \beta \in [0,1]
\]
Denoting \[
Z_{\beta} = \int_{\Theta} \pi_{\beta}^* (\theta) d \theta
\]We have the marginal likelihood $p(D) = Z_1$ while $Z_0 = \int_{\Theta} p(\theta) d\theta = 1$. The normalized distribution
\[
\pi_{\beta} (\theta) = \frac{\pi_{\beta}^*(\theta)}{Z_{\beta}}
\]Thermodynaimic integration is based on numeric integration
\[
\ln Z_1 - \ln Z_0 = \int_0^1 \frac{\partial \ln Z_{\beta}}{\partial \beta} d \beta
\]
which can be avaluated as 
\[
\ln p(D) = \ln Z_1 - \ln Z_0 = \int_0^1 E_{\beta}[\ln p(D | \theta)] d \beta
\]
where $E_{\beta}[]$ is an expectation over the annealed posterior $\pi_{\beta}(\theta)$ which can be evaluated using Monte Carlo samples from the corresponding chain. 

\subsection{Thermodynamic integration example}
\begin{minted}{Python}
import scipy.integrate
def lnormpdf(x, mu, sigma):
    return np.sum(-0.5*np.log(2*np.pi) - np.log(sigma) - 0.5 * (x-mu)**2/sigma**2)
 
npr.seed(42)
# Simulate n=100 points of normally distributed data about mu=0.5
n = 100
data = 0.5 + npr.normal(size=n)
 
# Set the prior parameters
sigma_x = 1.0
mu0 = 0.0
sigma0 = 3.0
likelihood = lambda mu: lnormpdf(data, mu, sigma_x)
prior = lambda mu: lnormpdf(mu, mu0, sigma0)
CHAINS = 20
ITERS = 10000
betas = np.concatenate((np.array([0.0]), np.logspace(-5, 0, CHAINS)))
xx = np.zeros((ITERS, CHAINS+1))
for i in range(CHAINS+1):
    xx[:,i] = mhsample1(np.zeros(1), ITERS, lambda x: betas[i]*likelihood(x)+prior(x),
                        lambda x: x + 0.2/np.sqrt(betas[i]+1e-3)*npr.normal(size=1))
                        
lls_ti = np.zeros(xx.shape)
for i in range(lls_ti.shape[0]):
    for j in range(lls_ti.shape[1]):
        lls_ti[i,j] = likelihood(xx[i,j])
ll = np.mean(lls_ti, 0)
plt.plot(betas, ll)
plt.xlabel(r'$\beta$')
plt.ylabel(r'$E_\beta[ \log p(\mathcal{D} | \theta)]$')
plt.show()

print('Marginal likelihood:',scipy.integrate.simps(ll, betas))
\end{minted}

\subsection{Exercises}
\subsubsection{Importance sampling}
Evaluate the expectations $E[x^2]$ and $E[x^4]$ over standard normal $N(0,1)$ using importance sampling with $Laplace(0,1)$ as the surrogate density. Then swap the roles

\begin{minted}{Python}
import numpy as np
import numpy.random as npr

def laplacetarget(x):
    return -np.log(2) - np.abs(x)

def normaltarget(x):
    return -0.5*np.log(2*np.pi)-0.5*(x**2)

npr.seed(42)
x = npr.laplace(size=10000)
print('E_Laplace[x^2] =', np.mean(x**2))
print('E_Laplace[x^4] =', np.mean(x**4))
print('E_N,importance[x^2] =', np.mean(x**2 * np.exp(normaltarget(x) - laplacetarget(x))))
print('E_N,importance[x^4] =', np.mean(x**4 * np.exp(normaltarget(x) - laplacetarget(x))))

y = npr.normal(size=10000)
print('E_N[x^2] =', np.mean(y**2))
print('E_N[x^4] =', np.mean(y**4))
print('E_Lap,importance[x^2] =', np.mean(y**2 * np.exp(laplacetarget(y) - normaltarget(y))))
print('E_Lap,importance[x^4] =', np.mean(y**4 * np.exp(laplacetarget(y) - normaltarget(y))))

\end{minted}

\subsubsection{Thermodynamic integration for marginal likelihood estimation}

\section{Automatic differentiation and optimisation}
Many problems in computational statistics involve the search or optimisation of a complicated function over a high dimensional space. Use derivatives and gradients of the function!

Automatic differentiation can be used to compute derivatives easily, exactly and efficiently.

\subsection{Derivatives of multivariate functions }
Let $f: \mathcal{R}^n \to \mathcal{R}^m$. When $m=1$ and $n>1$, the $n$-dimensional vector of partial derivatives measuring the change of the function in each coordinate direction is the gradient

\[
\nabla f(x) = \Bigg( \frac{\partial f}{\partial x_1}, \dots, \frac{\partial f}{\partial x_n} \Bigg)
\]
Geometrically, the gradient is a vector pointing to the direction where the function grows the fastest.

When both $n,m > 1$, the $m \times n$ matrix of partial derivatives is the Jacobian matrix J
\[
J_{ij} = \frac{\partial f_i}{\partial x_j}
\]
The Jacobian matrix defines a mapping which is locally the best linear approximation of f.

When $m=1$ and $n>1$, we can also define an $n \times n$ matrix $H$ of second derivatives called the Hessian matrix
\[
H_{ij} = \frac{\partial^2 f}{\partial x_i \partial x_j}
\]
Geometrically the Hessian matrix defines the curvature of the function at a point.

Gradients and other derivatives of a function are important because they enable efficient optimisation using for example Newton's method for minimisation. 

Newton's method for minimisation in higher dimensions
\[
x_{n+1} = x_n - [Hf(x_n)]^{-1}\nabla f(x_n)
\]

\subsection{Approaches to differentiation}
There are three main approaches for differentiation of a function $f:\mathcal{R}^n \to \mathcal{R}^m$

\begin{enumerate}
    \item Numerical differentiation: $f'(x) \approx \frac{f(x+h) - f(x)}{h}$
    \item Analytic differentiation
    \item Automatic differentiation
\end{enumerate}

Numerical differentiation provides limited accuracy, analytic differentiation requires lot of manual work, but automatic differentiation is a convenient computational solution that is easy, accurate and numerically stable. 

\subsection{Different modes of automatic differentiation}
There are two different modes: forward mode and backward mode. 

\subsection{Automatic differentiation in Python}
\subsubsection{Autograd}
\begin{minted}{Python}
import autograd.numpy as np
import autograd.numpy.linalg as npl
from autograd import grad
 
def f(x, a):
    return np.sin(a*x**3 - 2*x**2 - 4*x)
 
g = grad(f)
 
t = np.linspace(-2.2, 3.5, 50)
 
# grad() only works for functions with a scalar value so it cannot be
# used with this this f() when the input is a vector, therefore we
# need a for loop to compute the values
df = np.zeros(t.shape)
for i in range(len(t)):
    df[i] = g(t[i], 1.0)
plt.plot(t, f(t, 1.0), color='b', label=r'$f$')
plt.plot(t, df, color='g', label=r"$f'$")
plt.legend()
plt.show()
\end{minted}

It is important to use autograd.numpy and autograd.scipy!

Use floats instead of integers, 1.0 and not 1!

\subsubsection{PyTorch}
\begin{minted}{Python}
import torch
import numpy as np
import matplotlib.pyplot as plt
 
dtype = torch.double
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU
 
def f(x, a):
    return torch.cos(a*x**3 - 2*x**2 - 4*x)
 
t = torch.linspace(-2.2, 3.5, 50, device=device, dtype=dtype)
 
# variables used for gradient evaluation are created with requires_grad=True
x = torch.zeros(1, requires_grad=True, device=device, dtype=dtype)
 
# gradients again need to be evaluated 
df = np.zeros(t.shape)
for i in range(len(t)):
    # set the value of the input variable without changing its gradient
    # via .data attribute
    x.data = t[i]
    v = f(x, 1.0)
    # .backward() on function output computes gradients
    v.backward()
    # access gradients via .grad attribute
    df[i] = x.grad
    # gradient needs to be reset before computing the next value
    x.grad.zero_()
 
# Numpy arrays needed for plotting can be accessed via .numpy()
plt.plot(t.numpy(), f(t, 1.0).numpy(), color='b', label=r'$f$')
plt.plot(t.numpy(), df, color='g', label=r"$f'$")
plt.legend()
plt.show()
\end{minted}

\subsection{Gradient-based optimisation methods}
Approximations to Newton's method
\begin{enumerate}
    \item Quasi-Newton methods: BFGS, L-BFGS
    \item Conjugate gradient methods
    \item Gradient descent with plain
\end{enumerate}
Often BFGS and L-BFGS are good choices

\subsection{Stochastic gradient optimisation}
Many optimisation problems in statistics such as maximum likelihood estimation with independent identically distributed (iid) observations are of the form
\[
\textrm{min}F(\theta) = \sum_{i=1}^N f(\textbf{x}_i, \mathbf{\theta})
\]
When $N >> 1$, we can approximate the gradient by using a subset of the data $\{x_{s1}, \dots, x_{sM} \}$
\[
\nabla F (\theta) \approx \frac{N}{M} \sum_{i=1}^M \nabla f(x_{si}, \theta) = g(\theta)
\]
Iteratively updating 
\[
\theta_{n+1} = \theta_n - \eta_n g(\theta_n)
\]
will converge

\subsection{Optimisation with constraints}
Many optimisation problems have constraints, such as when optimising the standard deviation $\sigma$, $\sigma > 0$. Transformations

\subsection{Optimisation examples}
Let's consider maximum likelihood estimation of Student's t-distribution as an example.  

Our goal is to estimate the maximum likelihood location parameter and scale parameter of the Student's t-distribution from observations. Given the data set $D = (x_1, \dots, x_2)$, this can be written as the minimisation problem with the objective 
\[
f(\mathbf{\theta}) = \sum_{i=1}^n - \log p(x_i | \mathbf{\theta})
\]

\begin{minted}{Python}
import autograd.numpy as np
import autograd.numpy.random as npr
import autograd.scipy.stats as sst
import autograd
import scipy.optimize
 
# Generate a synthetic data set
x = npr.randn(100) + 0.5
print('data mean:', x.mean(), 'data std:', x.std())

def student_logpdf_autograd(x, m, s, nu):
    return sst.t.logpdf(x, nu, m, s)
 
def ltarget(theta):
    return -np.sum(student_logpdf_autograd(x,
    theta[0], np.exp(theta[1]), 5.0))
 
g = autograd.grad(ltarget)
 
theta_opt = scipy.optimize.minimize(ltarget, 
np.zeros(2), method='BFGS', jac = g)
print('location estimate:', theta_opt.x[0],
      'scale estimate:', np.exp(theta_opt.x[1]))
\end{minted}


\subsection{Exercises}
\subsubsection{Autograd tutorial}
\begin{minted}{Python}
# Example usage of Autograd

import autograd.numpy as np
from autograd import grad

def sigmoid(x):
    return 1/(1+np.exp(-x))

d_sigmoid = grad(sigmoid)
# Above d_sigmoid is now callable function that
#returns the derivative of sigmoid
print(d_sigmoid(0.0))
# See how it differs from the finite differences
h=1e-9
print( (sigmoid(h)-sigmoid(0.0))/(h) )
\end{minted}

\section{Variational inference}
MCMC methods approximate the posterior $p( \theta | D) $ through samples. Variational inference provides an alternative approach: fitting an approximation $q(\theta) \approx p(\theta | D)$ with a simple functional form, such as a suitable normal distribution, and casting the inference task as an optimisation problem.

Good approximations of posterior means, but often underestimate posterior variances. Variational inference is faster than MCMC.

\subsection{Variational inference basics}
Variational inference is based on fitting an approximation $q(\theta)$ to the posterior by minimising the Kullback-Leibler divergence

\[
D_{KL}(q(\theta) | |p(\theta)|D)) = E_{q(\theta)} \Bigg[\log \frac{q(\theta)}{p(\theta | D)} \Bigg]
\]
The Kullback-Leibler divergence is not a proper metric. Nevertheless it satisfies:
\[
D_{KL}(q || p) \leq 0 \forall q, p
\]
\[
D_{KL}(q || p) = 0, q = p
\]The exact $D_{KL}$ is difficult to compute, evidence lower-bound (ELBO) is used instead:
\[
L = E_{q(\theta)} \Bigg[ \log \frac{p(\theta, D)}{q(\theta)} \Bigg] = \log p(D) - D_{KL} (q(\theta) || p(\theta | D))
\]
Because $D_{KL} (q(\theta) || p(\theta | D)) \geq 0$, we have $L \leq \log p(D)$

\subsection{Classical variational inference}
Classical algorithms are based on cyclic optimisation of $L$ one variable at a time while keeping others fixed.

\subsection{Doubly stochastic variational inference (DSVI) and Automatic differentiation variational inference (ADVI)}
The algorithms use the reparametrisation trick

Gradients of the expectations 
\[
E_{q(\theta)}[*]
\]
with respect to a changing distribution such as 
\[
q(\theta) = N(\theta; \mu, \Sigma)
\]
can be turned to expectations with respect to a fixed distribution via reparametrisation
\[
\theta = L \eta + \mu
\]
where $\eta \sim N(0, I)$ and $L$ is the Cholesky factor of $\Sigma $ satisfying $LL^T = \Sigma$

The ELBO can now be written as 
\[
L = \int \phi (\eta) \log \frac{p(L \eta + \mu, D)|L|}{\phi(\eta)} d \eta
\]where $\phi(\eta)$ is the density of $\eta$ and $|L|$ comes from the coordinate transformation

We can thus write
\[
L = E_{\phi (\eta)} [\log p(L \eta + \mu, D)] + \log |C| + E_{\phi (\eta)}[- \log \phi (\eta)]
\]
Gradients\[
\nabla_{\mu} L = E_{\phi (\eta)} [\nabla_{\theta} \log p(\theta, D)]
\]
\[
\nabla_{L} L = E_{\phi (\eta)} [\nabla_{theta} \log p(\theta, D)]\times \eta^T + \Delta_L
\]
where $\Delta_L = \textrm{diag}(1/l_{11}, \dots, 1/l_{dd})$
Expectations with Monte Carlo, gradients with automatic differentiation

\subsection{Stochastic optimisation for variational inference}
We can maximise $L$ with stochastic gradient ascent
\[
\mu_{t+1} = \mu_t + p_t \tilde{\nabla}_{\mu}L(\mu_t, L_t)
\]
\[
L_{t+1} = L_t + p_t \tilde{\nabla}_{L}L(\mu_t, L_t)
\]

\subsection{Example}
The target joint distribution:
\[
\log p(D, \mu) = \sum_{i =1}^n \log N(x_i ; \mu, \sigma_x^2) + \log N(\mu; \mu_0, \sigma_0^2)
\]We use the normal approximation $q(\mu) = N(\mu'; \hat{\mu}, \tilde{\mu}^2)$ which can be reparametrised using $\mu = \hat{\mu} + \tilde{\mu} \eta_{\mu}$, $\eta_{\mu} \sim N(0,1)$

\subsubsection{Autograd}
\begin{minted}{Python}
import autograd.numpy as np
import autograd
import matplotlib.pyplot as plt
 
# Define the normal pdf. Note parameters: mean, standard deviation
def lnormpdf(x, mu, sigma):
    return -0.5*np.log(2*np.pi) - np.log(sigma) - 0.5 * (x-mu)**2/sigma**2
 
# Doubly stochastic variational inference
# Algorithm 1 of Titsias and Lázaro-Gredilla (2014)
def dsvi(m0, c0, gradient, sample_eta, rho0, t0 = 100, niters = 10000):
    """Doubly stochastic variational inference from
    Algorithm 1 of Titsias and Lázaro-Gredilla (2014)
    Arguments:
    m0: initial value of mean (scalar)
    c0: initial value of standard deviation (scalar)
    logjoint: function returning the value of the log-joint distribution p(X, theta)
    sample_eta: function sampling fixed parameters eta
    rho0: initial learning rate the rho_t = rho0 / (t0 + t)
    t0: t0 for the above (default: 100)
    niters: number of iterations (default: 10000)"""
    m = m0
    c = c0
    mhist = np.zeros(niters)
    chist = np.zeros(niters)
    for t in range(niters):
        eta = sample_eta()
        theta = c * eta + m
        g = gradient(theta)
        m = m + rho0 / (t0 + t) * g
        c = c + rho0 / (t0 + t) * (g * eta + 1/c)
        mhist[t] = m
        chist[t] = c
    return m, c, mhist, chist
 
# Simulate n=100 points of normally distributed data about mu=0.5
n = 100
data = 0.5 + npr.normal(size=n)
 
# Set the prior parameters
sigma_x = 1.0
mu0 = 0.0
sigma0 = 3.0
 
# Define the target log-pdf as a sum of likelihood and prior terms
def logjoint(mu, data=data, sigma_x=sigma_x, mu0=mu0, sigma0=sigma0):
    return lnormpdf(data, mu, sigma_x).sum() + lnormpdf(mu, mu0, sigma0)
 
m, c, mhist, chist = dsvi(1.0, 1.0, autograd.grad(logjoint), npr.normal, 0.1)
print(m, c)

plt.plot(mhist)
plt.plot(chist)
plt.show()
\end{minted}

\begin{minted}[breaklines]{Python}
tt = np.linspace(0.1, 0.9, 50)
m_post = sigma0**2 * np.sum(data) / (n*sigma0**2 + sigma_x**2)
s2_post = 1/(n/sigma_x**2 + 1/sigma0**2)
y = np.exp(lnormpdf(tt, m_post, np.sqrt(s2_post)))
plt.plot(tt, y, label='exact')
# Note: c is +/- sqrt(variance), need abs() to get standard deviation!
y_vi = np.exp(lnormpdf(tt, m, np.abs(c)))
plt.plot(tt, y_vi, label='variational approximation')
plt.legend()
plt.show()
\end{minted}

\subsubsection{PyTorch}

\begin{minted}{Python}
import torch
import math
 
dtype = torch.double
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU
 
# Define the normal pdf. Note parameters: mean, standard deviation
def lnormpdf(x, mu, sigma):
    return -0.5*math.log(2*math.pi) - torch.log(sigma) - 0.5 * (x-mu)**2/sigma**2
 
def dsvi(m0, c0, logjoint, sample_eta, rho0, t0 = 100, niters = 10000):
    """Doubly stochastic variational inference from
    Algorithm 1 of Titsias and Lázaro-Gredilla (2014)
    Arguments:
    m0: initial value of mean (tensor of length 1)
    c0: initial value of standard deviation (tensor of length 1)
    logjoint: function returning the value of the log-joint distribution p(X, theta)
    sample_eta: function sampling fixed parameters eta
    rho0: initial learning rate the rho_t = rho0 / (t0 + t)
    t0: t0 for the above (default: 100)
    niters: number of iterations (default: 10000)"""
    m = m0
    c = c0
    mhist = torch.zeros(niters, device=device, dtype=dtype)
    chist = torch.zeros(niters, device=device, dtype=dtype)
    for t in range(niters):
        eta = sample_eta()
        theta = torch.tensor(c * eta + m, requires_grad=True,
                             device=device, dtype=dtype)
        v = logjoint(theta)
        v.backward()
        g = theta.grad
        m = m + rho0 / (t0 + t) * g
        c = c + rho0 / (t0 + t) * (g * eta + 1/c)
        theta.grad.zero_()
        mhist[t] = m
        chist[t] = c
    return m, c, mhist, chist
# Set the seed for PyTorch RNGs
torch.manual_seed(42)
# Simulate n=100 points of normally distributed data about mu=0.5
n = 100
data = 0.5 + torch.randn(n, device=device, dtype=dtype)
 
# Set the prior parameters
sigma_x = torch.tensor(1.0, device=device, dtype=dtype)
mu0 = torch.tensor(0.0, device=device, dtype=dtype)
sigma0 = torch.tensor(3.0, device=device, dtype=dtype)
 
# Define the target log-pdf as a sum of likelihood and prior terms
def logjoint(mu, data=data, sigma_x=sigma_x, mu0=mu0, sigma0=sigma0):
    return lnormpdf(data, mu, sigma_x).sum() + lnormpdf(mu, mu0, sigma0)
 
m, c, mhist, chist = dsvi(torch.tensor(1.0, device=device, dtype=dtype),
                          torch.tensor(1.0, device=device, dtype=dtype),
                          logjoint,
                          lambda: torch.randn(1, device=device, dtype=dtype),
                          0.1)
print(m.item(), c.item())

plt.plot(mhist.numpy())
plt.plot(chist.numpy())
plt.show()
\end{minted}

\subsection{Exercises}
\subsubsection{Examining the fit of a variational approximation}
Target distribution is $p(\mu) = N(4, 2^2)$ and our approximation $q(\mu)$ is also normal

Parametrise  $\mu$  as  $μ=c⋅z+m$  and write a function to evaluate the negative Kullback-Leibler (KL) divergence
\[
L=−D_{KL}(q(μ)‖p(μ))=E_{q(μ)}[\log p(μ)−\log q(μ)]
\]
\[
=E_{\Phi (z)}[\log p(\mu)−\log \Phi(z)+\log |c|]
\]
Evaluate the expectation by sampling  $z \sim \Phi(z)=N(z;0,1) $. Evaluate the KL divergence for a range of values for  $m$  while keeping  $c=2$  fixed and for  c  while keeping  $m=4$  fixed, and plot the resulting values as a function of  m  and as a function of  c  (one line graph for each).

\begin{minted}{Python}
%matplotlib inline
import autograd.numpy as np
import autograd.numpy.random as npr
import matplotlib.pyplot as plt

#Log of normal
def lnormpdf(x, mu, sigma):
    return -0.5*np.log(2*np.pi*sigma**2) - 0.5*(x-mu)**2/sigma**2

#Kullback-Leibler divergence
def kl(m, c, N=1000):
    z = npr.normal(size=N)
    mu = c*z + m
    return np.mean(lnormpdf(mu, 4, 2) - lnormpdf(z, 0, 1) + np.log(np.abs(c)))

m = np.linspace(0, 8, 100)
y = np.array([kl(mm, 2) for mm in m])
plt.plot(m, y)
plt.show()
c = np.linspace(-1, 5, 100)
y = np.array([kl(4, cc) for cc in c])
plt.plot(c, y)
plt.show()
\end{minted}

\subsubsection{Fitting a variational approximation}
Use stochastic gradient optimisation to maximise  $L$  using Algorithm 1 of Titsias and Lázaro-Gredilla:

The function gradient(theta) here is assumed to return the gradient $ ∇\log p(θ)$  (or  $∇ \log p(θ,X)$  for posterior inference). The  $\log q(z)$  term can be ignored as it does not depend on  m  and  c .

In order for the stochastic gradient algorithm to converge, the step size sequence  $p_t$  should satisfy  $\sum_{t=1}^{\infty}p_t=1$ ,  $\sum_{t=1}^{\infty}p_t^2 < \infty$ . A common choice for such a sequence is 
\[
p_t = \frac{p_0}{t_0 + t}
\]which depends on parameters  $p_0$  and  $t_0$ that need to be tuned for each problem. You should try different values and plot the iterates $ m$  and  $c$  to see how they behave. Suitable  $p_0$  can vary significantly for different problems, while something like  $t_0=100$  might be a good starting point for many problems.

\begin{minted}{Python}
import autograd

#doubly stochastic variational inference
def dsvi(sample_z, m0, c0, gradient, rho_0, t0 = 100, niters = 10000):
    m = m0
    c = c0
    mhist = np.zeros(niters)
    chist = np.zeros(niters)
    for t in range(niters):
        z = sample_z()
        theta = c * z + m
        g = gradient(theta)
        m = m + rho_0 / (t0 + t) * g
        c = c + rho_0 / (t0 + t) * (g * z + 1/c)
        mhist[t] = m
        chist[t] = c
    return m, c, mhist, chist

def target(x):
    return lnormpdf(x, 4, 2)

m, c, mhist, chist = dsvi(npr.normal, 1.0, 1.0, autograd.grad(target), 10)
print(m, c)
plt.plot(mhist)
plt.plot(chist)
\end{minted}

\subsubsection{Variational inference for normal mean estimation from data}
Use doubly stochastic variational inference to approximate the posterior for  $μ$  for a normal model  $x_i∼N(μ,σ2)$  over the toy data with prior $p(μ)=N(μ_0,σ^20)$ where  $σ^2=2$,$μ_0=3$,$σ_0^2=10$ 

Parametrise  $μ$  as  $μ=c⋅z+m$  and write a function to evaluate the ELBO
\[
L=E_{q(μ)} [ \log p(X,μ)−\log q(μ)]=
\]
\[
E_{ϕ(z)}[\sum_{i=1}^n \log p(x_i|μ)+\log p(μ)−\log \phi(z)+\log |c|]
\]
by sampling  $z \sim \phi(z)=N(z;0,1) $.

Compare the result with  the exact posterior
$p(\mu | X, \sigma^2) = N(\mu; m_{exact}, v_{exact})$

\begin{minted}[breaklines]{Python}
import autograd.numpy as np
import autograd.numpy.random as npr
import pandas as pd
import scipy.special as scs

data = pd.read_csv('http://www.helsinki.fi/~ahonkela/teaching/compstats1/toydata2.txt', sep='\t', header=None)
data = data.values
data = np.array(data[:,0])

print(np.mean(data), np.std(data))

sigma0 = np.sqrt(10)
mu0 = 3
sigma = np.sqrt(2)
n = len(data)

def target2(mu):
    return np.sum(lnormpdf(data, mu, sigma)) + lnormpdf(mu, mu0, sigma0)

m, c, mhist, chist = dsvi(npr.normal, 1.0, 1.0, autograd.grad(target2), 0.5)
print(m, c)
v_exact = 1/(1/sigma0**2 + n/sigma**2)
m_exact = v_exact * (np.sum(data)/sigma**2 + mu0/sigma0**2)
print(m_exact, np.sqrt(v_exact))
plt.plot(mhist)
plt.plot(chist)
\end{minted}

\subsubsection{Variational approximation for a multimodal distribution}
Going back to the setting of Problem 1, repeat the fitting of a normal variational approximation  $q(x)$ to a target distribution that is a mixture of two normals:
$p(x)=1/2N(x;−R,1)+1/2N(x;R,1)$
 
1. Estimate the optimal variational approximation  $q(x)$  that follows a normal distribution for the above problem. Plot the convergence curve ( m  and  c  values as a function of the number of iterations) to make sure you are converging efficiently.

2. Run the estimation for several values of  $R$  in the range  $[0,5]$ . Plot the estimated mean and variance as a function of  $R$ . What do you observe?

3. Analyse the phase transition in the results. Plot the target distribution and the approximation for a few cases around the phase transition. Can you explain the transition?

\begin{minted}[breaklines]{Python}
import autograd.scipy.misc as scm

def target3(x, R):
    return scm.logsumexp(np.log(0.5) + np.array([lnormpdf(x, -R, 1), lnormpdf(x, R, 1)]))

rvals = np.linspace(0.1, 5, 25)
mvals = np.zeros(len(rvals))
cvals = np.zeros(len(rvals))
for i in range(len(rvals)):
    m, c, _, _ = dsvi(npr.normal, 1.0, 1.0, autograd.grad(lambda x: target3(x, rvals[i])), 10)
    mvals[i] = m
    cvals[i] = c
    print(rvals[i], m, c)
plt.plot(rvals, mvals)
plt.plot(rvals, cvals)

i = 24
m, c, mv, cv = dsvi(npr.normal, 1.0, 1.0, autograd.grad(lambda x: target3(x, rvals[i])), 10)
plt.plot(mv)
plt.plot(cv)

t = np.linspace(-8, 8, 100)
I = [14, 15]
for i in I:
    target_y = np.exp(np.array([target3(x, rvals[i]) for x in t]))
    approx_y = np.exp(lnormpdf(t, mvals[i], np.abs(cvals[i])))
    plt.plot(t, target_y, 'r')
    plt.plot(t, approx_y, 'b')
    plt.show()
\end{minted}

\section{Hamiltonian Monte Carlo HMC}
Using gradient information of the target function can help improve MCMC convergence. HMC is especially useful in high dimensions where a simple random walk does not converge easily. 

\subsection{Metropolis-adjusted Langevin algorithm MALA}
Metropolis-adjusted Langevin algorithm stimulates a reversible Langevin diffusion whose stationary distribution is the target $\pi(\theta)$:
\[
d \theta_t = \sigma d B_t + \frac{\sigma^2}{2} \nabla \log \pi (\theta_t) dt
\]This expression is a stochastic differential equation, where $B_t$ is the standard $d$-dimensional Brownian motion

Time discretisation:
\[
\theta'_{t+1} = \theta_t + \sigma_n z_{t+1} + \frac{\sigma_n^2}{2} \nabla \log \pi (\theta_t)
\]
where $z_{t+1} \sim N(0, I_d)$

The proposal is accepted as usual with probability
\[
\alpha (\theta_t, \theta'_{t+1} ) = \frac{\pi(\theta'_{t+1})}{\pi(\theta_t)} \frac{q(\theta_t; \theta'_{t+1})}{q(\theta'_{t+1}; \theta_t)}
\]
where 
\[
q(\theta'_{t+1}; \theta_t) = N \Bigg(\theta'_{t+1}; \theta_t + \frac{\sigma_n^2}{2} \nabla \log \pi (\theta_t), \sigma_n^2 I_d \Bigg)
\]The MALA algorithm itself is not very widely used in practice

\subsection{Hamiltonian Monte Carlo HMC}
Currently the most efficient general purpose samplers. Defining a dynamical system where $-\log \pi (\theta)$ represents the potential energy of the system which is then simulated using Hamiltonian dynamics.

For HMC we augment each regular position variable $\theta_i$ with a corresponding momentum variable $r_i \sim N(0,1)$. Denoting the potential energy defined by the target as $U(\theta) = - \log \pi (\theta)$, the momentum variables have a corresponding kinetic energy $K(r) = \sum_{i =1}^d \frac{1}{2}r_i^2$with total energy given by $H(\theta, r) = U(\theta) + K(r)$

Hamiltonian dynamics of the system
\[
\frac{d\theta_i}{dt} = \frac{\partial H(\theta, r)}{\partial r_i} = r_i
\]
\[
\frac{dr_i}{dt} = - \frac{\partial H(\theta, r)}{\partial \theta_i} = \frac{\partial \log \pi (\theta)}{\partial \theta_i}
\]\subsection{Numerical solution of Hamilton's dynamics}
Leapfrog integrator:
\[
r(t + \epsilon / 2) = r(t) - (\epsilon/2)\nabla_{\theta}U(\theta(t))
\]
\[
\theta(t + \epsilon ) = \theta(t) - \epsilon r(t + \epsilon/2)
\]
\[
r(t + \epsilon) = r(t + \epsilon/2) - (\epsilon/2)\nabla_{\theta}U(\theta(t + \epsilon))
\]\subsection{HMC algorithm}
Each step of the algorithm starts with simulating a normally distributed random momentum $r$. Given this momentum, a trajectory of $L$ leapfrog steps is simulated to obtain a new proposal pair $( \theta', r')$. The proposal pair is then either accepted or rejected according to MH acceptance rule in the augmented space of $(\theta, r)$. 
\[
\log a = H(\theta, r) - H(\theta', r')
\]
 \subsection{Tuning HMC}
HMC has two tunable parameters: $\epsilon$ and $L$

The effect of increasing $\epsilon$ can be quite dramatic. Acceptance rate of about 0.8 is in general a good target. 

\subsection{HMC theory}
Ultimately, the HMC sampler is just a Metropolis-Hastings sampler with a very complex proposal. HMC does not solve the problem of multiple modes! 

\subsection{Examples}
\[
\log \pi(\theta) = -20(||\theta||_2 - 10)^2
\]
\begin{minted}[breaklines]{Python}
import autograd.numpy as np
import autograd.numpy.random as npr
import autograd
import matplotlib.pyplot as plt
 
def hmc(theta0, M, target, epsilon, L):
    thetas = np.zeros([M, len(theta0)])
    gradF = autograd.grad(target)
    theta = np.copy(theta0)
    g = gradF(theta)  # set gradient using initial theta
    logP = target(theta)  # set objective function too
    accepts = 0
    for m in range(M): # draw M samples
        p = npr.normal(size=theta.shape)  # initial momentum is Normal(0,1)
        H = p.T @ p / 2 - logP   # evaluate H(theta,p)
        thetanew = np.copy(theta)
        gnew = np.copy(g)
        for l in range(L): # make L 'leapfrog' steps
            p = p + epsilon * gnew / 2   # make half-step in p
            thetanew = thetanew + epsilon * p    # make step in theta
            gnew = gradF(thetanew)           # find new gradient
            p = p + epsilon * gnew / 2   # make half-step in p
        logPnew = target(thetanew)   # find new value of H
        Hnew = p.T @ p / 2 - logPnew
        dH = H - Hnew    # Decide whether to accept
        if np.log(npr.rand()) < dH:
            g = gnew
            theta = thetanew
            logP = logPnew
            accepts += 1
        thetas[m,:] = theta
    print('Acceptance rate:', accepts/M)
    return thetas
 
def sphere(theta):
    return -20*(np.sqrt(np.sum(theta**2))-10)**2
 
samples = hmc(np.array([3.0, 0.0]), 200, lambda theta: sphere(theta), 0.2, 50)

samples = samples[samples.shape[0]//2:]
plt.plot(samples[:,0], samples[:,1])
plt.show()

import torch
import matplotlib.pyplot as plt
 
def sphere(theta):
    return -20*(torch.sqrt(torch.sum(theta**2))-10)**2
 
def hmc(theta0, M, target, epsilon, L):
    dtype = torch.double
    device = theta0.device
    thetas = torch.zeros([M, len(theta0)], device=device, dtype=dtype)
    theta = torch.tensor(theta0, requires_grad=True, device=device, dtype=dtype)
    logP = target(theta)  # set objective function too
    logP.backward()
    g = theta.grad
    accepts = 0
    for m in range(M): # draw M samples
        p = torch.normal(torch.zeros(theta.shape, device=device,
                                     dtype=dtype))  # initial momentum is Normal(0,1)
        H = p.dot(p) / 2 - logP   # evaluate H(theta,p)
        thetanew = torch.tensor(theta.data, requires_grad=True,
                            device=device, dtype=dtype)
        gnew = torch.tensor(g, device=device, dtype=dtype)
        for l in range(L): # make L 'leapfrog' steps
            p.data += epsilon * gnew.data / 2   # make half-step in p
            thetanew.data += epsilon * p.data          # make step in theta
            if thetanew.grad is not None:
                thetanew.grad.zero_()
            logPnew = target(thetanew)
            logPnew.backward()
            gnew = thetanew.grad             # find new gradient
            p.data += epsilon * gnew.data / 2   # make half-step in p
        logPnew = target(thetanew)   # find new value of H
        Hnew = p.dot(p) / 2 - logPnew
        dH = H - Hnew    # Decide whether to accept
        if torch.log(torch.rand(1, dtype=dtype, device=device)) < dH:
            g = gnew
            theta = thetanew
            logP = logPnew
            accepts += 1
        thetas[m,:] = theta.detach()
    print('Acceptance rate:', accepts/M)
    return thetas
 
# Set the seed for PyTorch RNGs
torch.manual_seed(42)
samples = hmc(torch.tensor([3.0, 0.0], device='cpu', dtype=torch.double),
              200, lambda theta: sphere(theta), 0.2, 50)
              
samples = samples[samples.shape[0]//2:]
plt.plot(samples.numpy()[:,0], samples.numpy()[:,1])
plt.show()  

\end{minted}

\subsection{Exercises}
We will consider the following examples as test cases:

1. Circle: $\log p_1(x) = -20*(||x||_2 - 10)^2$

2. Correlated 2-d normal: $p_2(x) = N(x; 0, \Sigma_2)$, where $\Sigma_2 = \begin {bmatrix} 1 & p \\ p & 1 \end{bmatrix}$with $p=0.998$

3. 20-d normal: $p_3(x) = N(x; 0, \Sigma_3)$, where $\Sigma_3 = diag(0.05^2, 0.1, \dots, 0.95^2, 1.00^2)$

\subsubsection{MCMC videos}
\begin{minted}[breaklines]{Python}
# Hint: many latter examples will use autograd so we will use autograd.numpy libraries throughout
%matplotlib inline
import autograd.numpy as np
import autograd.numpy.random as npr
import matplotlib.pyplot as plt

#Multidimensional MH sampler
def d_mhsample(x0, n, target, proposal, drawproposal, return_accrate=False):
    x = x0
    d = len(x0)
    accepts = 0
    lp = target(x)
    xs = np.zeros([n, d])
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        a = l_prop - lp + proposal(x_prop, x) - proposal(x, x_prop)
        if  0<a or npr.rand() < np.exp(a):
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    if return_accrate:
        return accepts/n
    else:
        print('Acceptance rate:', accepts/n)
        return xs[len(xs)//2:]
        
 import autograd.numpy.linalg as npl

def circle(x):
    return -20*(np.sqrt(x[0]**2+x[1]**2)-10)**2

def correlated_normal(x, rho=0.998):
    Sigma = np.array([[1.0, rho],[rho, 1.0]])
    return -0.5*(x.T @ npl.solve(Sigma, x))

def big_normal(x):
    return -0.5*np.sum(x**2 / (np.arange(1, 21)/20)**2)

samples1 = d_mhsample(np.zeros(2), 30000, circle, lambda x,y: 0, lambda x: x+npr.normal(size=2))
plt.plot(samples1[:,0], samples1[:,1], '.')
print(np.mean(samples1, 0), np.std(samples1, 0))

samples2 = d_mhsample(np.zeros(2), 30000, correlated_normal, lambda x,y: 0, lambda x: x+0.2*npr.normal(size=2))
plt.plot(samples2[:,0], samples2[:,1], '.')
print(np.mean(samples2, 0), np.std(samples2, 0))

samples3 = d_mhsample(np.zeros(20), 50000, big_normal, lambda x,y: 0, lambda x: x+0.1*npr.normal(size=20))
#plt.plot(samples3[:,0], samples3[:,1], '.')
plt.plot(np.arange(1, 21)/20, np.mean(samples3, 0))
plt.plot(np.arange(1, 21)/20, np.std(samples3, 0))
plt.xlabel('True standard deviation')
plt.ylabel('Estimated mean')
#print(np.mean(samples3, 0), np.std(samples3, 0))

# Handy trick for plotting the samples of the last example in parallel
plt.plot(samples3[::10] + 2*np.arange(20))

from IPython.display import HTML
from matplotlib import animation, rc

samples = samples1

# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim((-12, 12))
ax.set_ylim((-12, 12))

x = np.linspace(-12.0, 12.0, 100)
y = np.linspace(-12.0, 12.0, 100)
X, Y = np.meshgrid(x, y)

Z = np.zeros((len(x), len(y)))
# Need to use a for loop here because f() does not support vectorised evaluation
for i, x_i in enumerate(x):
    for j, y_j in enumerate(y):
        Z[i,j] = circle(np.array([x_i,y_j]))

plt.contour(X, Y, np.log(np.abs(Z.T)))
pts, = ax.plot([], [], 'r', lw=2)

# initialization function: plot the background of each frame
def init():
    pts.set_data([], [])
    return (pts,)

# animation function. This is called sequentially
def animate(i):
    pts.set_data(samples[np.max([0, 10*i-500]):10*i:5,0], samples[np.max([0, 10*i-500]):10*i:5,1])
    return (pts,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=1000, interval=20, blit=True)

HTML(anim.to_html5_video())
\end{minted}

\subsubsection{HMC sampling}
\begin{minted}[breaklines]{Python}
%matplotlib inline
import autograd.numpy as np
import autograd.numpy.random as npr
import autograd
import matplotlib.pyplot as plt

#HMC sampler
def hmc(theta0, M, target, epsilon0, L):
    thetas = np.zeros([M, len(theta0)])
    gradF = autograd.grad(target)
    theta = np.copy(theta0)
    g = gradF(theta)  # set gradient using initial theta
    logP = target(theta)  # set objective function too
    accepts = 0
    for m in range(2*M): # draw M samples after M warm-up iterations
        p = npr.normal(size=theta.shape)  # initial momentum is Normal(0,1)
        H = p.T @ p / 2 - logP   # evaluate H(x,p)
        thetanew = np.copy(theta)
        gnew = np.copy(g)
        for l in range(L): # make L 'leapfrog' steps
            epsilon = npr.uniform(0.8, 1.2) * epsilon0  # optional: randomise epsilon for improved theoretical convergence properties
            p = p + epsilon * gnew / 2   # make half-step in p
            thetanew = thetanew + epsilon * p    # make step in theta
            gnew = gradF(thetanew)           # find new gradient
            p = p + epsilon * gnew / 2   # make half-step in p
        logPnew = target(thetanew)   # find new value of H
        Hnew = p.T @ p / 2 - logPnew
        dH = Hnew - H    # Decide whether to accept
        if np.log(npr.rand()) < -dH:
            g = gnew
            theta = thetanew
            logP = logPnew
            accepts += 1
        if m >= M:
            thetas[m-M,:] = theta
    print('Acceptance rate:', accepts/(2*M))
    return thetas
    
#Circle sample
samples1 = hmc(np.array([2.0, 0.0]), 500, lambda theta: circle(theta), 0.2, 10)
plt.plot(samples1[:,0], samples1[:,1], '.')
print(np.mean(samples1, 0), np.std(samples1, 0))

#Correlated normal
samples2 = hmc(np.array([3.0, 0.0]), 100, correlated_normal, 0.07, 30)
plt.plot(samples2[:,0], samples2[:,1], '.')
print(np.mean(samples2, 0), np.std(samples2, 0))

samples3 = hmc(np.zeros(20), 2000, big_normal, 0.05, 20)
#plt.plot(samples3[:,0], samples3[:,1], '.')
#print(np.mean(samples3, 0), np.std(samples3, 0))
plt.plot(np.arange(1, 21)/20, np.mean(samples3, 0))
plt.plot(np.arange(1, 21)/20, np.std(samples3, 0))
plt.xlabel('True variance')
plt.ylabel('Estimated mean')

plt.plot(samples3 + 2*np.arange(20))

from IPython.display import HTML
from matplotlib import animation, rc

samples = samples1

# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim((-12, 12))
ax.set_ylim((-12, 12))

x = np.linspace(-12.0, 12.0, 100)
y = np.linspace(-12.0, 12.0, 100)
X, Y = np.meshgrid(x, y)

Z = np.zeros((len(x), len(y)))
# Need to use a for loop here because f() does not support vectorised evaluation
for i, x_i in enumerate(x):
    for j, y_j in enumerate(y):
        Z[i,j] = circle(np.array([x_i,y_j]))

plt.contour(X, Y, np.log(np.abs(Z.T)))
pts, = ax.plot([], [], 'r', lw=2)

# initialization function: plot the background of each frame
def init():
    pts.set_data([], [])
    return (pts,)

# animation function. This is called sequentially
def animate(i):
    pts.set_data(samples[np.max([0, i-50]):i,0], samples[np.max([0, i-50]):i,1])
    return (pts,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=1000, interval=20, blit=True)

HTML(anim.to_html5_video())
\end{minted}

\subsubsection{MALA sampling}
\begin{minted}[breaklines]{Python}
import autograd.numpy as np
from autograd import grad

def lnormpdf(x, y, sigma):
    n = len(x)
    return n/2*np.log(2*np.pi*sigma**2) - 0.5/sigma**2*np.sum((x-y)**2)

def malasample(theta0, n, target, sigma, return_accrate=False):
    dtarget = grad(target)
    theta = theta0
    d = len(theta0)
    accepts = 0
    lp = target(theta)
    g_theta = dtarget(theta)
    thetas = np.zeros([n, d])
    for i in range(n):
        theta_prop = theta + sigma*npr.normal(size=d) + sigma**2/2 * g_theta
        l_prop = target(theta_prop)
        g_theta_prop = dtarget(theta_prop)
        a = (l_prop - lp 
             + lnormpdf(theta,      theta_prop + sigma**2/2 * g_theta_prop, sigma)
             - lnormpdf(theta_prop, theta      + sigma**2/2 * g_theta,      sigma))
        if  np.log(npr.rand()) < a:
            theta = theta_prop
            lp = l_prop
            g_theta = g_theta_prop
            accepts += 1
        thetas[i] = theta
    if return_accrate:
        return accepts/n
    else:
        print('Acceptance rate:', accepts/n)
        return thetas[len(thetas)//2:]


samples = malasample(np.array([0.0, 6.0]), 30000, circle, 0.3)
plt.plot(samples[:,0], samples[:,1], '.')

# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim((-12, 12))
ax.set_ylim((-12, 12))

x = np.linspace(-12.0, 12.0, 100)
y = np.linspace(-12.0, 12.0, 100)
X, Y = np.meshgrid(x, y)

Z = np.zeros((len(x), len(y)))
# Need to use a for loop here because f() does not support vectorised evaluation
for i, x_i in enumerate(x):
    for j, y_j in enumerate(y):
        Z[i,j] = circle(np.array([x_i,y_j]))

plt.contour(X, Y, np.log(np.abs(Z.T)))
pts, = ax.plot([], [], 'r', lw=2)

# initialization function: plot the background of each frame
def init():
    pts.set_data([], [])
    return (pts,)

# animation function. This is called sequentially
def animate(i):
    pts.set_data(samples[np.max([0, 3*i-300]):3*i, 0], samples[np.max([0, 3*i-300]):3*i, 1])
    return (pts,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=1000, interval=20, blit=True)

HTML(anim.to_html5_video())

samples = malasample(np.array([0.0, 10.0]), 10000, circle, 0.3)
plt.plot(samples[:,0], samples[:,1], '.')

samples = malasample(np.array([0.0, 0.0]), 10000, correlated_normal, 0.13)
plt.plot(samples[:,0], samples[:,1], '.')
print(np.mean(samples, 0), np.std(samples, 0))

samples3 = malasample(np.zeros(20), 50000, big_normal, 0.1)
print(samples3.shape)
plt.plot(np.arange(1, 21)/20, np.mean(samples3, 0))
plt.plot(np.arange(1, 21)/20, np.std(samples3, 0))
plt.xlabel('True variance')
#plt.ylabel('Estimated mean')

plt.plot(samples3[::10] + 2*np.arange(20))
\end{minted}

\section{Dynamic Hamiltonian Monte Carlo and No-U-Turn Sampling (NUTS)}
The biggest problem in standard HMC is the choice of the number of steps $L$ for the Hamiltonian system simulation. HMC is an all-or-nothing game. 

\subsection{No-U-Turn Sampling (NUTS)}
No-U-Turn Sampler (NUTS) was the first proposed dynamic HMC algorithm. NUTS extends the simulated trajectory until it observes a U-turn of the path turning back on itself. 

\subsection{NUTS $\epsilon$ tuning}
$\epsilon$ is tuned by first doubling or halving the value to find an  $ϵ$ that gives Langevin proposal acceptance rate of approximately 0.5.

\subsection{NUTS sampling}
The sampling iteration in NUTS differs from HMC in two respects:
\begin{enumerate}
\item The simulated trajectory or path in the Hamiltonian system is extended dynamically both forward and backward until a termination condition designed to detect a U-turn is met
\item The next point is sampled from valid alternatives generated during above extension step instead of a single accept/reject choice.
\end{enumerate}



\subsection{Exercises}
\subsubsection{NUTS}
Redo the examplkes from the previous section (HMC)
\begin{minted}[breaklines]{Python}
# Hint: many latter examples will use autograd so we will use autograd.numpy libraries throughout
%matplotlib inline
import autograd.numpy as np
import autograd.numpy.random as npr
import matplotlib.pyplot as plt
import nuts

def d_mhsample(x0, n, target, proposal, drawproposal, return_accrate=False):
    x = x0
    d = len(x0)
    accepts = 0
    lp = target(x)
    xs = np.zeros([n, d])
    for i in range(n):
        x_prop = drawproposal(x)
        l_prop = target(x_prop)
        a = l_prop - lp + proposal(x_prop, x) - proposal(x, x_prop)
        if  0<a or npr.rand() < np.exp(a):
            x = x_prop
            lp = l_prop
            accepts += 1
        xs[i] = x
    if return_accrate:
        return accepts/n
    else:
        print('Acceptance rate:', accepts/n)
        return xs[len(xs)//2:]
        
import autograd.numpy.linalg as npl

def circle(x):
    return -20*(np.sqrt(x[0]**2+x[1]**2)-10)**2

def correlated_normal(x, rho=0.998):
    Sigma = np.array([[1.0, rho],[rho, 1.0]])
    return -0.5*(x.T @ npl.solve(Sigma, x))

def big_normal(x):
    return -0.5*np.sum(x**2 / (np.arange(1, 21)/20)**2)
    
import nuts

samples1 = nuts.nuts6(circle, 100, 100, np.array([6.0, 0.0]))
plt.plot(samples1[:,0], samples1[:,1], '.')
print(np.mean(samples1, 0), np.std(samples1, 0))

import nuts

samples = nuts.nuts6(correlated_normal, 1000, 100, np.zeros(2))
plt.plot(samples[:,0], samples[:,1], '.')
print(np.mean(samples, 0), np.std(samples, 0))

import nuts

samples3 = nuts.nuts6(big_normal, 10000, 1000, np.zeros(20))
#plt.plot(samples3[:,0], samples3[:,1], '.')
plt.plot(np.arange(1, 21)/20, np.mean(samples3, 0))
plt.plot(np.arange(1, 21)/20, np.std(samples3, 0))
plt.xlabel('True standard deviation')
plt.ylabel('Estimated mean + standard deviation')
#print(np.mean(samples3, 0), np.std(samples3, 0))

p = plt.plot(samples3[::5] + 2*np.arange(20))
\end{minted}

\subsubsection{HMC sampling for posterior inference in linear regression}
Problem 4 from Automatic differentiation

\begin{minted}[breaklines]{Python}
import pandas as pd
import autograd.numpy as np
import autograd
from scipy.optimize import minimize
import nuts

def standardize(x):
    return (x - np.mean(x)) / (2*np.std(x))

# load the data from CSV file using pandas
fram = pd.read_csv('http://www.helsinki.fi/~ahonkela/teaching/compstats1/fram.txt', sep='\t')
# convert the variables of interest to numpy arrays for autograd compatibility
# input: Framingham relative weight - the ratio of the subjects weight to the median weight for their sex-height group
x = np.array(fram['FRW'])
# target: Systolic blood pressure, examination 1
y = np.array(fram['SBP'])

xs = standardize(x)
ys = standardize(y)

def linear_regression_logl(coefs, x, y):
    return np.sum(-0.5*np.log(2*np.pi)-0.5*(y - coefs[0] - coefs[1] * x)**2)

def linear_regression(x, y):
    myfun = lambda c: -linear_regression_logl(c, np.array(x), np.array(y))
    dmyfun = autograd.grad(myfun)
    v = minimize(myfun, np.ones(2), jac=dmyfun)
    return v

def laplace_regression_logl(coefs, x, y):
    return np.sum(-np.log(2)-np.abs(y - coefs[0] - coefs[1] * x))

def laplace_regression(x, y):
    myfun = lambda c: -laplace_regression_logl(c, np.array(x), np.array(y))
    dmyfun = autograd.grad(myfun)
    v = minimize(myfun, np.ones(2), jac=dmyfun, method='L-BFGS-B')
    return v

def normal_logp(x, mu, sigma):
    return np.sum(-0.5*np.log(2*np.pi*sigma**2) - 0.5*((x-mu)**2/sigma**2))

def norm_linreg_target(theta, x, y):
    return linear_regression_logl(theta, xs, ys) + normal_logp(theta[0], 0, 2) + normal_logp(theta[1], 0, 2)

def lap_linreg_target(theta, x, y):
    return laplace_regression_logl(theta, xs, ys) + normal_logp(theta[0], 0, 2) + normal_logp(theta[1], 0, 2)

samples_norm = nuts.nuts6(lambda theta: norm_linreg_target(theta, xs, ys), 1000, 200, np.array([1.0, 0.0]))
print(np.mean(samples_norm, 0), np.std(samples_norm, 0))
print(linear_regression(xs, ys)['x'])

import nuts
samples_lap = nuts.nuts6(lambda theta: lap_linreg_target(theta, xs, ys), 1000, 200, np.array([1.0, 0.0]))
print(np.mean(samples_lap, 0), np.std(samples_lap, 0))
print(laplace_regression(xs, ys)['x'])
\end{minted}

\subsubsection{HMC sampling for posterior inference 2}
Apply HMC/NUTS to perform Bayesian inference over the mean $\mu$ and the standard deviation $\sigma$ in a normal model
 \[
 p(x_i | \mu, \sigma^2) = N(x_i; \mu, \sigma^2)
 \]
 \[
 p(\mu) = N(\mu; \mu_0, \sigma_0^2)
 \]
\[
p(\sigma^2) = InvGamma(\alpha, \beta)
\]
\begin{minted}[breaklines]{Python}
%matplotlib inline
import autograd.numpy as np
import autograd.numpy.random as npr
import pandas as pd
import autograd.scipy.special as scs
import matplotlib.pyplot as plt
import nuts

data = pd.read_csv('http://www.helsinki.fi/~ahonkela/teaching/compstats1/toydata2.txt', sep='\t', header=None)
data = data.values
data = np.array(data[:,0])

def lnorm(x, mu, sigma2):
    return np.sum(-0.5*np.log(2*np.pi*sigma2) - 0.5*(x-mu)**2/sigma2)

def linvgamma(x, alpha, beta):
    return np.sum(alpha*np.log(beta) - scs.gammaln(alpha) - (alpha+1)*np.log(x) - beta/x)

def linvgamma_t(y, alpha, beta):
    return linvgamma(np.exp(y), alpha, beta) + np.sum(y)

def target(theta, mu0, sigma0, alpha, beta):
    mu = theta[0]
    sigma2 = np.exp(np.min([theta[1], 30]))
    return lnorm(data, mu, sigma2) + lnorm(mu, mu0, sigma0) + linvgamma_t(sigma2, alpha, beta)

print(np.mean(data), np.var(data))
samples = nuts.nuts6(lambda theta: target(theta, 0.0, 100.0, 2.0, 2.0), 1000, 200, np.zeros(2))
samples[:,1] = np.exp(samples[:,1])
print(np.mean(samples, 0), np.std(samples, 0))
\end{minted}

\section{Likelihood-free inference and approximate Bayesian computation ABC}
MCMC and variational inference are general inference methods for models where we can compute the likelihood $p(D|\theta)$. 

Some models are computationally intractable!

Likelihood-free inference methods are based on simulating new synthetic data sets. The inference is based on comparing the simulated synthetic data sets with the observed data set

In all model-based statistical inference, the $\textbm{likelihood function}$ is of central importance, since it expresses the probability of the observed data under a particular statistical model, and thus quantifies the support data lend to particular values of parameters and to choices among different models. For simple models, an analytical formula for the likelihood function can typically be derived. However, for more complex models, an analytical formula might be elusive or the likelihood function might be computationally very costly to evaluate.

\subsection{Approximate Bayesian computation}
The idea of ABC is simple
\begin{enumerate}
    \item Sample $\theta^*$ from the prior $p(\theta)$
    \item Simulate a new observed data set $D^*$ conditional to $\theta^*$: $D^* = \eta(\theta^*) ~ p(D|\theta^*)$
    \item Accept or reject the sample according to criteria specified below (Exact rejection ABC or Summary statistics ABC)
\end{enumerate}

\subsection{Exact rejection ABC}
A simulated sample $\theta^*$ is accepted if and only if $D^* = D$. 

\subsection{Summary statistics ABC}
ABC can be made more efficient by requiring that only some summary statistics $S(D)$ should match between the generated and the observed data sets. E.g. means and variances of the data. 

We can accept proposals with 
\[
|| S(D) - S(D^*) || < \epsilon
\]
for some $\epsilon > 0$. 

\subsection{What is approximate ABC sampling from?}
The above approximate procedure of accepting samples with $||D - D^*|| < \epsilon $ can be generalised to a procedure where the proposed sample $\theta^*$ is accepted with probability 
\[
a = \frac{\pi_{\epsilon}(D - D^*)}{c}
\]
where $\pi_{\epsilon}()$ is a suitable probability density and the constant $c$ is chosen to guarantee that $\pi_{\epsilon}(D - D^*)/c$ defines a probability. 
Applying ABC with this acceptance probability can be shown to be equivalent to sampling from the posterior distribution of the model.

\subsection{Further developments}
Methods coupling ABC with some form of MCMC or otherwise learning where to sample the proposals can help.

ABC bears clear conceptual similarity with generative adversarial networks (GANs) that are widely used in machine learning with deep neural networks. GAN learning is based on a game where the generator attempts to generate synthetic samples and the discriminator tries to distinguish these. 

\subsection{Exercises}
\subsubsection{Exact rejection ABC}
Implement an exact rejection ABC sampler to sample from the posterior distribution of the probability $p$ of obtaining heads in a coin flip when observing $r$ heads in $n$ throws:
\[
p \sim \textrm{Uniform(0,1)}, r \sim \textrm{Binom(n,p)}
\]
In other words, $\textrm{Uniform(0,1)}$ is the prior distribution for the parameter $p$.

1. Simulate 1000 samples from the posterior when $n = 20$, $r = 10$.

\begin{minted}{Python}
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

#N is the number of samples
#n throws
#r heads

def abc_coinflip(N, n, r):
    samples = np.zeros(N)
    nsamples = 0
    while nsamples < N:
        #Simulate a probability p    
        p = npr.uniform()
        #p* is accepted if D*=D
        if npr.binomial(n, p) == r:
            samples[nsamples] = p
            nsamples += 1
    return samples

x = abc_coinflip(1000, 20, 10)
h = plt.hist(x, 30)
\end{minted}

2. Simulate 1000 samples from the posterior when $n = 40$, $r = 25$ and estimate the probability $p < 0,5$.

\begin{minted}{Python}
x = abc_coinflip(1000, 40, 25)
h = plt.hist(x, 30)
print(np.mean(x < 0.5))
\end{minted}

3. Simulate 1000 samples from the posterior when $n = 80$, $r = 50$ and estimate the probability $p < 0,5$.

\begin{minted}{Python}
x = abc_coinflip(1000, 80, 50)
h = plt.hist(x, 30)
print(np.mean(x < 0.5))
\end{minted}

\subsubsection{ABC for a discrete switching time series}
We will consider a Markov model with two states 0 and 1. The model is
\[
p(x_{t+1} | x_t) = \begin{cases}
1 - \pi, & if x_{t+1} = x_t\\
\pi, & if x_{t+1} \neq x_t
\end{cases}
\]
i.e. the state will flip with probability $\pi$. We will assume the chain starts in state $x_0 = 0$.

We set a uniform prior $p(\pi) = Uniform( \pi; 0,1)$

1. Implement an exact ABC sampler that generates full sequences and accepts if the full sequences match. Test your sampler by generating at least 100 samples from the posterior distribution given a single observed sequence "0011001100".

\begin{minted}{Python}
#N is the number of samples
def abc_switching_exact(N, seq):
    samples = np.zeros(N)
    nsamples = 0
    n = len(seq)
    while nsamples < N:
        p = npr.uniform()
        i = 1
        accept = True
        while i < n and accept:
            if seq[i] == seq[i-1]:
                if npr.uniform() < p:
                    accept = False
            else:
                if npr.uniform() > p:
                    accept = False
            i += 1
        if accept:
            samples[nsamples] = p
            nsamples += 1
    return samples

#x = abc_switching_exact(10000, '01010101010101010101')
#h = plt.hist(x, 30)
%time x = abc_switching_exact(10000, '0011001100')
h = plt.hist(x, 30)
print(np.mean(x), np.std(x))
\end{minted}

2. Implement an ABC sampler that uses the number of state flips $S(X) = #\{i | x_i \neq x_{i+1}\}$ as the sufficient statistic. Are the two samplers sampling from the same distribution? How long sequences can each of them handle?

\begin{minted}{Python}
def abc_switching_summary(N, n, switches):
    samples = np.zeros(N)
    nsamples = 0
    while nsamples < N:
        p = npr.uniform()
        if npr.binomial(n, p) == switches:
            samples[nsamples] = p
            nsamples += 1
    return samples

%time y = abc_switching_summary(10000, 9, 4)
h = plt.hist(y, 30)
print(np.mean(y), np.std(y))
\end{minted}

3. Extend your sampler from subtask 2 by allowing acceptance when $|S(X) - S(X^*)| \leq \epsilon$. Test this sampler and the one from subtask 2 for a sequence of length 200 that always flips state after 2 symbols, i.e. "00110011...0011". Check how the accuracy of the posterior mean and standard deviation, and the computing time depend on $\epsilon$.

\begin{minted}{Python}
y = abc_switching_summary(10000, 199, 99)
h = plt.hist(y, 30)
print(np.mean(y), np.std(y))

def abc_switching_summary_approx(N, n, switches, epsilon):
    samples = np.zeros(N)
    nsamples = 0
    while nsamples < N:
        p = npr.uniform()
        if np.abs(npr.binomial(n, p) - switches) <= epsilon:
            samples[nsamples] = p
            nsamples += 1
    return samples

z = abc_switching_summary_approx(10000, 199, 99, 3)
h = plt.hist(z, 30)
print(np.mean(z), np.std(z))
\end{minted}

\subsubsection{Rejection ABC with continuous data}
Infer the posterior for $\mu$ and $\sigma^2$ for a normal model $x_i ~ N(\mu, \sigma^2)$ over the toy data with priors 
\[
p(\mu) = N(4, \sqrt{10}^2), p(\sigma^2) = Gamma(2,2)
\]
We will use the mean and standard deviation of $x_i$ as the summary statistics $S(x)$ and accept samples if $||S(x^*) - S(x)|| < \epsilon$

1. Generate 1000 samples from the prior, evaluate the errors in the summary statistics and plot their distribution

\begin{minted}{Python}
import numpy as np
import numpy.random as npr
import pandas as pd

def compute_summaries(x):
    return np.array([np.mean(x), np.std(x)])

data = pd.read_csv('http://www.helsinki.fi/~ahonkela/teaching/compstats1/toydata.txt', sep='\t', header=None)
data = data.values
data = np.array(data[:,0])
datasummaries = compute_summaries(data)

mu0 = 4.0
sigma0 = np.sqrt(10)
alpha0 = 2.0
beta0 = 2.0

def normal_abc_errors(N, epsilon = np.inf, Ndata=len(data)):
    samples = np.zeros((N, 2))
    errors = np.zeros(N)
    nsamples = 0
    while nsamples < N:
        m = npr.normal(mu0, sigma0)
        v = npr.gamma(alpha0, 1/beta0)
        mydata = npr.normal(m, np.sqrt(v), Ndata)
        myerr = np.sqrt(np.sum((datasummaries - compute_summaries(mydata))**2))
        if myerr <= epsilon:
            samples[nsamples,0] = m
            samples[nsamples,1] = v
            errors[nsamples] = myerr
            nsamples += 1
    return samples, errors

x, errs = normal_abc_errors(1000)
h = plt.hist(errs, 30)
\end{minted}

2. Compute the approximate posterior distributions using $\epsilon = 3$ and $\epsilon = 1$. Compare their means and standard deviations.

\begin{minted}{Python}
x3, errs = normal_abc_errors(1000, 3.0)
x = x3
h = plt.hist(x[:,0], 30)
plt.show()
h = plt.hist(x[:,1], 30)

x1, errs = normal_abc_errors(1000, 1.0)
x = x1
h = plt.hist(x[:,0], 30)
plt.show()
h = plt.hist(x[:,1], 30)
\end{minted}